# Code for analysis and figures of disk packings

By Jessica H. Sun and Abigail Plummer

Licensed under the terms of GNU GENERAL PUBLIC LICENSE by Free Software Foundation

Change parent_directory variable to point to location of dataverse_files folder, which can be downloaded from [Harvard Dataverse](https://doi.org/10.7910/DVN/ZQ4BUR).

In [ ]:
parent_directory='/Users/jessicasun/Downloads/'

This notebook is used to generate main text figures and psi6 cutoff plot in Fig S3.

In [ ]:
import zipfile
import io
import matplotlib as mpl
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import sympy
import seaborn as sns
import time
from scipy.stats import circmean
import pickle
import numpy as np
import pandas as pd
import os
from os import system
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D 
import mpl_toolkits.mplot3d.art3d as art3d
from math import atan2, cos, sin

In [ ]:
input_dir=parent_directory+'dataverse_files/022322 bennett simulation_datfiles.zip' #path to parent directory containing /datfiles subdirectory, or a .zip of the parent directory
simulation_input1='run_dir_list_022322_0-99.npy.zip'
simulation_input2='run_dir_list_022322_0-99_cylinders0-5cinit.npy'

def unzip_input(file_input): #unzip if needed, then unpack .npy or .pkl
    if file_input.endswith('.zip'):
        inputzip = zipfile.ZipFile(file_input,'r')
        inputzip_bytes = io.BytesIO(inputzip.read(inputzip.infolist()[0]))
        if file_input.endswith('.pkl.zip'):
            file_output=pickle.load(inputzip_bytes)        
        if file_input.endswith('.npy.zip'):
            file_output=np.load(inputzip_bytes)            
    else:
        if file_input.endswith('.pkl'):
            with open(file_input, "rb") as fh:            
                file_output=pickle.load(fh)
        if file_input.endswith('.npy'):
            file_output=np.load(file_input)            
    return file_output

a=unzip_input(simulation_input1)
b=unzip_input(simulation_input2)
run_dir_list=np.unique(np.concatenate((a,b)))

if not os.path.exists(os.getcwd()+'/analysis'):
    os.mkdir(os.getcwd()+'/analysis')

In [ ]:
def get_params_dict(run_dir):
    keys=[]
    values=[]
    for params in run_dir.split('_'):
        key,value=params.split('-')
        keys+=[key]
        values+=[float(value)]        
    params_dict=dict(zip(keys,values))
    if params_dict['cthetadeg']==0:
        params_dict['run_dir']=run_dir
        params_dict['yinit']=params_dict['cinit']/(0.002) #from simulation definition
        params_dict['R']=params_dict['yinit']+20 #from simulation definition
        params_dict['thetadeg']=0
    else:
        params_dict['run_dir']=run_dir
        params_dict['yinit']=params_dict['cinit']/(params_dict['cthetadeg']*np.pi/180) #from simulation definition
        params_dict['R']=params_dict['yinit']+20 #from simulation definition
        params_dict['thetadeg']=2*ctheta_to_alpha(params_dict['cthetadeg']*np.pi/180)*180/np.pi
    return params_dict
def get_run_dir_df(run_dir_list):
    run_dir_df_list=[]
    for run_dir in run_dir_list:
        params_dict=get_params_dict(run_dir)
        run_dir_df_list+=[pd.DataFrame([params_dict])]
    run_dir_df=pd.concat(run_dir_df_list,axis=0)
    return run_dir_df
def run_dir_df_info(run_dir_df,input_dir=input_dir,output_dir=os.getcwd()+'/analysis'):
    print('input_dir','\n',input_dir)
    print('\noutput_dir','\n',output_dir,'\n')
    info_list=[]
    for item in run_dir_df.columns[-1:]:
        print(item,'\n',np.unique(run_dir_df[item]),'\n')
        info_list+=[str(item)+'-'+str(int(np.min(np.unique(run_dir_df[item]))))+'-'+str(int(np.max(np.unique(run_dir_df[item]))))]
    for item in run_dir_df.columns[:5]:
        print(item,'\n',np.unique(run_dir_df[item]),'\n')
        info_list+=[str(item)+'-'+str(int(np.min(np.unique(run_dir_df[item]))))+'-'+str(int(np.max(np.unique(run_dir_df[item]))))]
    info_tag='_'.join(info_list)
    print('info_tag','\n',info_tag,'\n')
    return info_tag
def ctheta_to_alpha(cthetarad):
    alpharad=np.arcsin(cthetarad/(2*np.pi))
    return alpharad
def load_dat(run_dir,input_dir=input_dir):
    if input_dir[-4:]=='.zip':
        input_type='zip'
    else:
        input_type='dir'
    if input_type=='dir': #simple read in of data coordinates and adj matrix
        particles=pd.read_csv('datfiles/'+run_dir+'/particles.dat', sep=' ', header=None).to_numpy()
        adj=pd.read_csv('datfiles/'+run_dir+'/adj.dat', sep=' ', header=None).to_numpy()
    if input_type=='zip':
        if os.path.exists(input_dir[:-2]+'01'): #if split archive
            if not os.path.exists('data-full.zip'):
                system('zip -FF "'+input_dir+'" --out data-full.zip') #automatically finds any additional split archive files like .z01 and concats into a full zip
            inputzip = zipfile.ZipFile('data-full.zip','r')
        else: #not a split archive; directly load .zip
            inputzip = zipfile.ZipFile(input_dir,'r')
        path_datfiles=inputzip.namelist()[0]    
        particles = pd.read_csv(io.BytesIO(inputzip.read(path_datfiles+run_dir+'/particles.dat')), sep=' ', header=None).to_numpy()
        adj = pd.read_csv(io.BytesIO(inputzip.read(path_datfiles+run_dir+'/adj.dat')), sep=' ', header=None).to_numpy()           
    return particles,adj
def find_coordnum(particles,adj):
    energy_list=[]
    for particle_idx in range(len(particles)):
        energy=sum(adj[:,particle_idx])
        energy_list+=[energy]
    data=pd.concat([pd.DataFrame(particles,columns=['x','y']),pd.DataFrame(energy_list,columns=['coordnum'])],axis=1)    
    return data,adj
def load_data(run_dir,shifted=True,input_dir=input_dir): #shifted moves the seed to the side so that seam is likely at center
    particles,adj=load_dat(run_dir,input_dir=input_dir)
    data,adj=find_coordnum(particles,adj)
    if shifted:
        data=get_data_shifted(data,4/3,params_dict['cthetadeg'])        
    return data,adj
def load_data_slice(run_dir,n,shifted=True): #number of particles to load (indices are in order of placement)
    particles,adj=load_dat(run_dir)    
    particles,adj=particles[:n],adj[:n]
    data_slice,adj=find_coordnum(particles,adj)
    if shifted:
        data_slice=get_data_shifted(data_slice,4/3,params_dict['cthetadeg'])        
    return data_slice,adj
def rotate_point_shift(r,theta,rotatethetadeg):
    rotated_theta=theta+rotatethetadeg*np.pi/180
    y=r*np.sin(rotated_theta)
    x=r*np.cos(rotated_theta)
    return x,y,rotated_theta
def plot_data_shifted(data_shifted):
    fig=plt.figure(figsize=(15,15))
    ax=fig.add_subplot(131)
    ax.scatter(data_shifted['x'],data_shifted['y'])
    ax.scatter(data_shifted['x'][:3],data_shifted['y'][:3])
    ax.set_aspect('equal')
    ax.invert_yaxis()    
    ax.set_title('unshifted data')

    ax=fig.add_subplot(132)
    ax.scatter(data_shifted['x_rotated'],data_shifted['y_rotated'])
    ax.scatter(data_shifted['x_rotated'][:3],data_shifted['y_rotated'][:3])
    ax.set_aspect('equal')
    ax.invert_yaxis()    
    ax.set_title('rotated data')

    ax=fig.add_subplot(133)
    ax.scatter(data_shifted['x_final'],data_shifted['y_final'])
    ax.scatter(data_shifted['x_final'][:3],data_shifted['y_final'][:3])
    ax.set_aspect('equal')
    ax.invert_yaxis()
    ax.set_title('final shifted data')
def get_data_shifted(data, distfromedge,cthetadeg,plot_data_shifted_bool=False):
    if params_dict['cthetadeg']!=0:
        rotatethetadeg=(params_dict['cthetadeg']/2)-((distfromedge)/(params_dict['cinit']/(params_dict['cthetadeg']*np.pi/180)))*180/np.pi #(4/3)/(cinit/(cthetadeg*np.pi/180)) ->4/3 is arclength distance from edge, can be adjusted for display purposes
        data_shifted=pd.DataFrame([])
        data_shifted['x']=data['x']
        data_shifted['y']=data['y']
        data_shifted['r']=np.sqrt(data_shifted['x']**2+data_shifted['y']**2)
        data_shifted['theta']=np.arctan2(data_shifted['y'],data_shifted['x'])

        ctheta=cthetadeg*np.pi/180
        data_shifted['x_rotated'],data_shifted['y_rotated'],data_shifted['theta_rotated']=rotate_point_shift(data_shifted['r'],data_shifted['theta'],rotatethetadeg)

        data_shifted['theta_final']=np.zeros(len(data_shifted['theta_rotated'])) #initialize column
        data_shifted['x_final']=np.zeros(len(data_shifted['theta_rotated'])) #initialize column
        data_shifted['y_final']=np.zeros(len(data_shifted['theta_rotated'])) #initialize column
        for idx in range(len(data_shifted)):
            angle=data_shifted['theta_rotated'][idx]
            r=data_shifted['r'][idx]
            if angle>(-np.pi/2.0+ctheta/2.0):
                angle=angle-np.floor((np.abs(angle-(-np.pi/2))+ctheta*0.5)/ctheta)*ctheta
            if angle<=(-np.pi/2.0-ctheta/2.0):
                angle=angle+np.floor((np.abs(angle-(-np.pi/2))+ctheta*0.5)/ctheta)*ctheta
            data_shifted['theta_final'][idx]=angle
            data_shifted['x_final'][idx]=r*cos(angle)
            data_shifted['y_final'][idx]=r*sin(angle)
        if plot_data_shifted_bool:
            plot_data_shifted(data_shifted)
        data_shifted_reformat=pd.DataFrame([])
        data_shifted_reformat['x']=data_shifted['x_final']
        data_shifted_reformat['y']=data_shifted['y_final']
        data_shifted_reformat['coordnum']=data['coordnum']
    if params_dict['cthetadeg']==0:
        Lx=params_dict['cinit']
        rotatethetadeg=(Lx/2)-(distfromedge)
        data_shifted=pd.DataFrame([])
        data_shifted['x']=data['x']
        data_shifted['y']=data['y']

        data_shifted['x_rotated']=data_shifted['x']+rotatethetadeg
        data_shifted['y_rotated']=data_shifted['y']

        data_shifted['x_final']=np.zeros(len(data_shifted['x'])) #initialize column
        data_shifted['y_final']=np.zeros(len(data_shifted['x'])) #initialize column
        for idx in range(len(data_shifted)):            
            x,y=data_shifted['x_rotated'][idx],data_shifted['y_rotated'][idx]
            if x>Lx/2.0:
                x=x-np.floor((x+Lx*0.5)/(Lx))*Lx
            if x<=-Lx/2.0:
                x=x+np.floor(abs(x-Lx*0.5)/Lx)*Lx   
            data_shifted['x_final'][idx]=x
            data_shifted['y_final'][idx]=y
        if plot_data_shifted_bool:
            plot_data_shifted(data_shifted)
        data_shifted_reformat=pd.DataFrame([])
        data_shifted_reformat['x']=data_shifted['x_final']
        data_shifted_reformat['y']=data_shifted['y_final']
        data_shifted_reformat['coordnum']=data['coordnum']        
    return data_shifted_reformat

print('get run_dir_df...')
run_dir_df=get_run_dir_df(run_dir_list)
run_dir_df_info(run_dir_df)
run_dir_df.head()

print('done!')

In [ ]:
def get_newcmp(origcmap='plasma',newcmapname='newcmp',cyclic=False,crop=False,bottom=0,top=245,replace=True,r=100,g=100,b=100,a=1):
    newcolors = cm.get_cmap(origcmap, 256)(np.linspace(0, 1, 256))
    if crop:
        newcolors=newcolors[bottom:top,:]
    if replace:
        if not cyclic:
            newcolors[-25:, :] = np.array([r/256, g/256, b/256, a]) #165, 140, 0, 0.8
        if cyclic:
            newcolors[-25:, :] = np.array([r/256, g/256, b/256, a]) #165, 140, 0, 0.8
            newcolors[:25, :] = np.array([r/256, g/256, b/256, a]) #165, 140, 0, 0.8        
    newcmp = ListedColormap(newcolors)
    plt.register_cmap(name=newcmapname,cmap=newcmp)    
    return newcmp
def standalone_cbar(cmap='RdBu',orientation='vertical',discrete=False,n_clusters=5,vmin=0,vmax=6,title='title'):
    fig = plt.figure(figsize=(2,2),dpi=400)
    if orientation=='vertical':
        ax = fig.add_axes([0.05, 0.80, 0.05, 0.9]) #vertical [left bottom width height]
    if orientation=='horizontal':
        ax = fig.add_axes([0.05, 0.80, 0.9, 0.05]) #horizontal [left bottom width height]   
    if not discrete:
        cmap=plt.get_cmap(cmap)
        cb = mpl.colorbar.ColorbarBase(ax, orientation=orientation,cmap=cmap,norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax))
    if discrete:
        n_clusters=(vmax-vmin)+1
        cmap=plt.get_cmap(cmap, n_clusters)
        cb = mpl.colorbar.ColorbarBase(ax, orientation=orientation,cmap=cmap,norm=mpl.colors.Normalize(vmin=vmin, vmax=vmax))
        tick_locs = (np.arange(vmin,vmax,(vmax-vmin)/n_clusters))+0.5*((vmax-vmin)/n_clusters) #tick_locs def changed upon update, so dont include last point
        cb.set_ticks(tick_locs)
        cb.set_ticklabels(np.arange(vmin,vmax+1,1))
    cb.outline.set_visible(False)
    cb.ax.tick_params(size=0)
    cb.ax.set_title(title)
    return cb.get_ticks()

newcmp=get_newcmp(origcmap='plasma',newcmapname='newcmp',cyclic=False,r=180,g=180,b=180,a=0.4) #for sector plots
get_newcmp(origcmap='viridis',newcmapname='newviridis',cyclic=False,crop=True,bottom=0,top=225,replace=False,r=100,g=100,b=100,a=1) #for line plots
get_newcmp(origcmap='cividis',newcmapname='newcividis',cyclic=False,crop=True,bottom=0,top=225,replace=False,r=100,g=100,b=100,a=1) #for line plots

standalone_cbar(cmap='newcmp',orientation='horizontal',discrete=True,vmin=2,vmax=6,title=r'$N_{j}$')
plt.savefig(output_dir+'/newcmp_horizontal.svg',bbox_inches = "tight")
standalone_cbar(cmap='newcmp',orientation='vertical',discrete=True,vmin=2,vmax=6,title=r'$N_{j}$')
plt.savefig(output_dir+'/newcmp_vertical.svg',bbox_inches = "tight")

def get_dark_palette(cmp_str,index,ref_n,new_n,reverse): #ref_n is number of discrete colors in reference colormap. new_n is number of new colors in new colormap.
    palette=sns.color_palette(
            sns.dark_palette(
                mpl.colors.to_hex(cm.get_cmap(cmp_str,ref_n)(index)),new_n,reverse=reverse)[:new_n])
    return palette
def get_light_palette(cmp_str,index,ref_n,new_n,reverse): #ref_n is number of discrete colors in reference colormap. new_n is number of new colors in new colormap.
    palette=sns.color_palette(sns.light_palette(mpl.colors.to_hex(cm.get_cmap(cmp_str,ref_n)(index)),new_n,reverse=reverse)[:new_n])
    return palette

new_n=7
ref_n=5
theta_palette=sns.color_palette((get_dark_palette('newcmp',0,ref_n,new_n,False)[:-1]+get_light_palette('newcmp',0,ref_n,new_n,True))[::-1])
phi_palette=sns.color_palette((get_dark_palette('newcmp',1,ref_n,new_n,False)[:-1]+get_light_palette('newcmp',1,ref_n,new_n,True))[::-1])
c_palette=sns.color_palette((get_dark_palette('newcmp',2,ref_n,new_n,False)[:-1]+get_light_palette('newcmp',2,ref_n,new_n,True)))
psi_palette=sns.color_palette((get_dark_palette('newcmp',3,ref_n,new_n,False)[:-1]+get_light_palette('newcmp',3,ref_n,new_n,True)))

In [ ]:
def uv_wrap(x2D,y2D,ctheta_rad):
    if ctheta_rad!=0:
        x3D=[]
        y3D=[]
        z3D=[]
        for x,y in zip(x2D,y2D): #x,y describe coordinates of flat net of particles
            u,v=[(atan2(y,x)-np.pi/2+ctheta_rad/2)*(2*np.pi/ctheta_rad),np.sqrt(x**2+y**2)] #theta from (0 to +ctheta) then normalized and mapped onto 2pi, r distance from origin
            arclength=v*ctheta_rad #s=r*theta. arclength of sector is circumference of wrapped cone at given point
            cone_r=(arclength/np.pi)/2.0 #radius of 3D CONE, not the radial coord of flat net sector.
            cone_alpha=np.arcsin(ctheta_rad/(2*np.pi)) #alpha is half cone angle (angle of slant from cone vertical). use cone equation from first slide of prof nelson's pdf.
            cone_z=cone_r/np.tan(cone_alpha) #find height of cone. cone_r is radius, v is slant length.             
            x3D+=[cone_r*np.cos(u)] #convert u,v to x,y,z.
            y3D+=[cone_r*np.sin(u)] #convert u,v to x,y,z.
            z3D+=[cone_z] #convert u,v to x,y,z.
    if ctheta_rad==0:
        Lx=params_dict['cinit']
        x3D=[]
        y3D=[]
        z3D=[]
        for x,y in zip(x2D,y2D): #x,y describe coordinates of flat net of particles
            u=(x/Lx)*2*np.pi-np.pi/2
            v=-y            
            arclength=Lx #s=r*theta. arclength of sector is circumference of wrapped cone at given point
            cone_r=(arclength/np.pi)/2.0 #radius of 3D CONE, not the radial coord of flat net sector.
            cone_z=v #find height of cone. cone_r is radius, v is slant length.             
            x3D+=[cone_r*np.cos(u)] #convert u,v to x,y,z.
            y3D+=[cone_r*np.sin(u)] #convert u,v to x,y,z.
            z3D+=[cone_z] #convert u,v to x,y,z.        
    return x3D,y3D,z3D
def plot_sector(data,params_dict,ax,alpha=1,c='coordnum',cmap='viridis'):
    if c=='coordnum':
        vmin,vmax=2,6
    if c=='psi6':
        vmin,vmax=0,1
    if c=='theta6':
        vmin,vmax=0,60
    c=data[c]
    if params_dict['cthetadeg']!=0:
        ax.add_patch(patches.Wedge((0,0), -params_dict['R'], 90-params_dict['cthetadeg']/2, 90+params_dict['cthetadeg']/2,fill=False,edgecolor='black',linewidth=0.25))
    if params_dict['cthetadeg']==0:
        Lx=params_dict['cinit']
        Ly=params_dict['R']
        ax.add_patch(patches.Rectangle((-Lx/2,0),Lx,-params_dict['R'],fill=False,edgecolor='black',linewidth=0.25))        
    ax.scatter(data['x'],data['y'],c=c,s=1,alpha=alpha,cmap=cmap,vmin=vmin,vmax=vmax) #plot points
    try:
        ax.scatter(data.loc[0]['x'],data.loc[0]['y'],color='black',s=1,alpha=alpha,zorder=200) #plot seed
        ax.scatter(data.loc[1]['x'],data.loc[1]['y'],color='black',s=1,alpha=alpha,zorder=200) #plot seed
        ax.scatter(data.loc[2]['x'],data.loc[2]['y'],color='black',s=1,alpha=alpha,zorder=200) #plot seed    
    except:
        pass
    ax.set_xlim((np.min(data['x'])-10,np.max(data['x'])+10))
    ax.set_ylim((np.min(data['y'])-10,np.max(data['y'])+10))
    ax.set_aspect('equal')
    ax.set_title(run_dir+'\n'+'thetadeg-'+str(round(params_dict['thetadeg'],1))+'_yinit-'+str(round(params_dict['yinit'],1))+'_R-'+str(round(params_dict['R'],1)),fontsize=5)
    ax.axis('off')
    ax.invert_yaxis()
def plot_cone(data,params_dict,ax,planealpha=0.85,elev_val=0,azim_val=180,alpha=1,cmap='viridis'):
    zz,yy = np.meshgrid(range(int(np.min(data['z3D'])-10),int(np.max(data['z3D'])+10)), range(int(np.min(data['y3D'])-10),int(np.max(data['y3D'])+10)))     #plot semitransparent plane. we do this instead of setting alpha of point markers because matplotlib does a weird gradient for transparent markers
    xx = yy*0
    ax.plot_surface(xx, yy, zz,alpha=planealpha,color='w',shade=False,antialiased=False,linewidth=0,edgecolor=None,rstride=100,cstride=100) #if the alpha set for overall figure is 1, have a semitransparent plane. otherwise set the plane alpha value to half of that of the datapoint alpha value (input).    
    ax.set_xlim((np.min(data['x3D'])-2,np.max(data['x3D'])+2)) #set axis limits based on data bounds
    ax.set_ylim((np.min(data['y3D'])-2,np.max(data['y3D'])+2))
    ax.set_zlim((np.min(data['z3D'])-2,np.max(data['z3D'])+2))    
    ax.set_box_aspect((ax.get_xlim()[1]-ax.get_xlim()[0],ax.get_ylim()[1]-ax.get_ylim()[0],ax.get_zlim()[1]-ax.get_zlim()[0]))
    data_back=data[data['x3D']>=0] #back of 3D cone
    data_front=data[data['x3D']<0] #front of 3D cone
    data=data_back
    data_list=[(data_back,alpha*1), (data_front,alpha)] #alpha controls POINTS alpha
    for data,alpha in data_list:
        try:
            ax.scatter3D(data.loc[3:]['x3D'],data.loc[3:]['y3D'],data.loc[3:]['z3D'],c=data.loc[3:]['coordnum'],s=1,alpha=alpha,cmap=cmap) #plot points
            ax.scatter3D(data.loc[0:2]['x3D'],data.loc[0:2]['y3D'],data.loc[0:2]['z3D'],color='black',s=1,alpha=alpha) #plot seed               
        except:
            continue
    ax.set_title(run_dir+'\n'+'thetadeg-'+str(round(params_dict['thetadeg'],1))+'_yinit-'+str(round(params_dict['yinit'],1))+'_R-'+str(round(params_dict['R'],1)),fontsize=5)
    ax.axis('off')
    ax.invert_yaxis()
    ax.invert_xaxis()    
    ax.view_init(elev=elev_val,azim=azim_val)      
def plot_seed(data,params_dict,alpha=1,c='coordnum',cmap='viridis'):
    fig=plt.figure(figsize=(1.5,1.5),dpi=400)
    ax=fig.add_subplot(111)    
    data=data[:3] #take only seeds of the dataset
    seed0x, seed0y=data.loc[0]['x'],data.loc[0]['y']
    seed1x, seed1y=data.loc[1]['x'],data.loc[1]['y']
    seed2x, seed2y=data.loc[2]['x'],data.loc[2]['y']
    if c=='coordnum':
        vmin,vmax=2,6
    if c=='psi6':
        vmin,vmax=0,1
    if c=='theta6':
        vmin,vmax=0,60
    c=data[c]
    seedR=np.sqrt(seed0x**2+seed0y**2)
    try:
        ax.scatter(seed0x,seed0y,color='black',s=50,alpha=alpha) #plot seed
        ax.scatter(seed1x,seed1y,color='black',s=50,alpha=alpha) #plot seed
        ax.scatter(seed2x,seed2y,color='black',s=50,alpha=alpha) #plot seed    
        ellipsewidth=5*(seed1x-seed0x)
        ellipseheight=0.5
        ellipse=mpl.patches.Ellipse((seed0x,seed0y-ellipseheight/2),ellipsewidth,ellipseheight, angle=0,zorder=0,edgecolor='0.3',linewidth=1,facecolor='none')        
        ax.add_patch(ellipse)
        ax.annotate("", xy=(2*(seed1x-seed0x)+seed0x,2*(seed1y-seed0y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow
        ax.annotate("", xy=(-2*(seed1x-seed0x)+seed0x,-2*(seed1y-seed0y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow        
        ax.annotate("", xy=(2*(seed2x-seed0x)+seed0x,2*(seed2y-seed0y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow
        ax.annotate("", xy=(-2*(seed2x-seed0x)+seed0x,-2*(seed2y-seed0y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow        
        ax.annotate("", xy=(2*(seed2x-seed1x)+seed0x,2*(seed2y-seed1y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow
        ax.annotate("", xy=(-2*(seed2x-seed1x)+seed0x,-2*(seed2y-seed1y)+seed0y), xytext=(seed0x, seed0y),arrowprops=dict(arrowstyle="->",linewidth=0.5)) #plot arrow                
    except:
        pass
    ax.set_xlim(seed0x-3,seed0x+3)
    ax.set_ylim((seed0y-3),seed0y+3)
    ax.set_aspect('equal')
    ax.set_title(run_dir+'\n'+'thetadeg-'+str(round(params_dict['thetadeg'],1))+'_yinit-'+str(round(params_dict['yinit'],1))+'_R-'+str(round(params_dict['R'],1)),fontsize=5)
    ax.axis('off')
    ax.invert_yaxis()
    
cthetadeg=10
cinit=12
ithetadeg=40
trial=0

run_dir_df_filtered=run_dir_df[
    (run_dir_df['cthetadeg']==cthetadeg)
    &(run_dir_df['cinit']==cinit)
    &(run_dir_df['ithetadeg']==ithetadeg)
    &(run_dir_df['trial']==trial)]

run_dir=run_dir_df_filtered['run_dir'].iloc[0]
params_dict=get_params_dict(run_dir)
data,adj=load_data(run_dir)
data['x3D'],data['y3D'],data['z3D']=uv_wrap(data['x'],data['y'],params_dict['cthetadeg']*np.pi/180)

In [ ]:
cmap=newcmp

def get_reference_range(cthetadeg=0,cinit=np.unique(run_dir_df[(run_dir_df['cinit']>8.6)&(run_dir_df['cinit']<8.7)]['cinit'])[0],ithetadeg=30,trial=0):
    run_dir_df_filtered=run_dir_df[
        (run_dir_df['cthetadeg']==cthetadeg)
        &(run_dir_df['cinit']==cinit)
        &(run_dir_df['ithetadeg']==ithetadeg)
        &(run_dir_df['trial']==trial)]

    run_dir=run_dir_df_filtered['run_dir'][0]
    params_dict=get_params_dict(run_dir)
    data,adj=load_data(run_dir)
    datarange=max(data['y'])-min(data['y'])
    return datarange
reference_range=get_reference_range(cthetadeg=0,cinit=np.unique(run_dir_df[(run_dir_df['cinit']>8.6)&(run_dir_df['cinit']<8.7)]['cinit'])[0],ithetadeg=30,trial=0)
print('reference_range set')

def plot_arc(data,params_dict,ax,alpha=1,c='coordnum',cmap='viridis'):
    data=data[:3] #take only seeds of the dataset
    seed0x, seed0y=data.loc[0]['x'],data.loc[0]['y']
    seed1x, seed1y=data.loc[1]['x'],data.loc[1]['y']
    seed2x, seed2y=data.loc[2]['x'],data.loc[2]['y']
    if c=='coordnum':
        vmin,vmax=2,6
    if c=='psi6':
        vmin,vmax=0,1
    if c=='theta6':
        vmin,vmax=0,60
    c=data[c]
    seedR=np.sqrt(seed0x**2+seed0y**2)
    ax.add_patch(patches.Arc((0,0), -2*seedR,-2*seedR, angle=0,theta1=90-params_dict['cthetadeg']/2, theta2=90+params_dict['cthetadeg']/2,fill=False,edgecolor='0.3',linestyle='-',linewidth=1))
def plot_circ(data,params_dict,ax,alpha=1,c='coordnum',cmap='viridis'): #don't know why this doesn't work
    data=data[:3] #take only seeds of the dataset
    seed0x, seed0y=data.loc[0]['x'],data.loc[0]['y']
    seed1x, seed1y=data.loc[1]['x'],data.loc[1]['y']
    seed2x, seed2y=data.loc[2]['x'],data.loc[2]['y']
    
    if c=='coordnum':
        vmin,vmax=2,6
    if c=='psi6':
        vmin,vmax=0,1
    if c=='theta6':
        vmin,vmax=0,60
    c=data[c]
    seedR=np.sqrt(seed0x**2+seed0y**2)
    cone_r=(params_dict['cinit']/(2*np.pi))
    ctheta_rad=params_dict['cthetadeg']*np.pi/180
    cone_alpha=np.arcsin(ctheta_rad/(2*np.pi)) #alpha is half cone angle (angle of slant from cone vertical). use cone equation from first slide of prof nelson's pdf.
    cone_z=cone_r/np.tan(cone_alpha)    
    circle_theta=np.linspace(-3*np.pi/2,-np.pi/2,100)
    circle_x=cone_r*np.cos(circle_theta)
    circle_y=cone_r*np.sin(circle_theta)
    circle_z=np.repeat(cone_z,len(circle_theta))
    ax.plot(circle_x,circle_y,circle_z,color='0.3',zorder=15,linewidth=0.8) #circumference circle
    circle_theta=np.linspace(-np.pi/2,np.pi/2,100)
    circle_x=cone_r*np.cos(circle_theta)
    circle_y=cone_r*np.sin(circle_theta)
    circle_z=np.repeat(cone_z,len(circle_theta))
    ax.plot(circle_x,circle_y,circle_z,color='0.3',zorder=15,linewidth=0.8,alpha=0.73) #circumference circle
    ax.plot(data.loc[0:2]['x3D'],data.loc[0:2]['y3D'],data.loc[0:2]['z3D'],'ko',markersize=1,color='black',alpha=alpha,zorder=15) #plot seed                   

In [ ]:
reference_range=get_reference_range(cthetadeg=0,cinit=np.unique(run_dir_df[(run_dir_df['cinit']>8.6)&(run_dir_df['cinit']<8.7)]['cinit'])[0],ithetadeg=30,trial=0)
print('reference_range set')
trial_list=[90] # trial_list=[12,17,23,33,56,79,90]
for trial in trial_list:
    cthetadeg=20
    cinit=12
    ithetadeg=30
    trial=trial
    run_dir_df_filtered=run_dir_df[
        (run_dir_df['cthetadeg']==cthetadeg)
        &(run_dir_df['cinit']==cinit)
        &(run_dir_df['ithetadeg']==ithetadeg)
        &(run_dir_df['trial']==trial)]
    run_dir=run_dir_df_filtered['run_dir'].iloc[0]
    params_dict=get_params_dict(run_dir)
    data,adj=load_data(run_dir,shifted=False)
    data['x3D'],data['y3D'],data['z3D']=uv_wrap(data['x'],data['y'],params_dict['cthetadeg']*np.pi/180)
    fig=plt.figure(figsize=(6.5,6.5),dpi=400)
    ax=fig.add_subplot(111)
    plot_sector(data,params_dict,ax,cmap=cmap)
    plot_arc(data,params_dict,ax,cmap=cmap)
    Nactual=len(data)
    info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)
    ax.set_ylim(min(data['y'])-10,min(data['y'])+reference_range+50)
    ax.invert_yaxis()
    # plt.savefig(output_dir+'/fig2a'+info_tag+'.png')
    plt.savefig('analysis/fig1sector.svg',bbox_inches = "tight")

    fig=plt.figure(figsize=(3,3),dpi=400)
    ax=fig.add_subplot(111,projection='3d')
    plot_cone(data,params_dict,ax,azim_val=180,cmap=cmap)
    plot_circ(data,params_dict,ax,alpha=1,c='coordnum',cmap='viridis')
    plot_cone(data,params_dict,ax,alpha=0,planealpha=0,azim_val=180,elev_val=10) #set bounds
#     plt.savefig(output_dir+'/fig2b'+info_tag+'.png')
    plt.savefig('analysis/fig1cone.svg',bbox_inches = "tight")

    ###########
    plot_seed(data,params_dict)
    plt.savefig('analysis/fig1seed.svg',bbox_inches = "tight")
    ############
    plt.show()

In [ ]:
def rotate_point(r,theta,rotatethetadeg):
    y=r*np.sin(theta+rotatethetadeg*np.pi/180)
    x=r*np.cos(theta+rotatethetadeg*np.pi/180)
    return x,y
def find_images(data,params_dict):
    data['r']=np.sqrt(data['x']**2+data['y']**2)
    data['theta']=np.arctan2(data['y'],data['x'])
    if params_dict['cthetadeg']!=0:
        data['circ']=data['r']*params_dict['cthetadeg']*np.pi/180
        data['x_left'],data['y_left']=rotate_point(data['r'],data['theta'],-params_dict['cthetadeg'])
        data['x_right'],data['y_right']=rotate_point(data['r'],data['theta'],+params_dict['cthetadeg'])
    if params_dict['cthetadeg']==0:
        Lx=params_dict['cinit']
        data['circ']=Lx
        data['x_left'],data['y_left']=data['x']-Lx,data['y']
        data['x_right'],data['y_right']=data['x']+Lx,data['y']
    return data
def select_image(particle, neighbor):
    distsquared=(neighbor['x']-particle['x'])**2+(neighbor['y']-particle['y'])**2
    distsquared_left=(neighbor['x_left']-particle['x'])**2+(neighbor['y_left']-particle['y'])**2
    distsquared_right=(neighbor['x_right']-particle['x'])**2+(neighbor['y_right']-particle['y'])**2
    distsquared_list=[distsquared,distsquared_left,distsquared_right]
    distsquared_min=min(distsquared_list)
    if distsquared_list.index(distsquared_min)==0:
        return neighbor['x'],neighbor['y']
    if distsquared_list.index(distsquared_min)==1:
        return neighbor['x_left'],neighbor['y_left']
    if distsquared_list.index(distsquared_min)==2:
        return neighbor['x_right'],neighbor['y_right']
def get_data_neighbors(particle_idx,data,adj,run_dir):
    particle=data.loc[particle_idx,:]
    neighbor_idx_list=np.nonzero(adj[particle_idx])[0]
    data_neighbors_list=[]
    for neighbor_idx in neighbor_idx_list:
        neighbor=data.loc[neighbor_idx,:].copy()
        neighbor['select_x'],neighbor['select_y']=select_image(particle,neighbor)
        data_neighbors_list+=[neighbor]
    data_neighbors=pd.concat(data_neighbors_list,axis=1).T.reset_index()
    return data_neighbors
def get_theta6_psi6(data,adj,run_dir):
    for particle_idx in range(len(data)):
        angle_deg_list=[]
        psi6_list=[]
        vector_list=[]
        data_neighbors=get_data_neighbors(particle_idx,data,adj,run_dir) #find positions of neighboring particles (including the image position)
        particle=data.loc[particle_idx,:]
        for neighbor_idx in range(len(data_neighbors)):
            neighbor=data_neighbors.iloc[neighbor_idx,:] #select by row number not actual index label
            x,y=particle['x'],particle['y']
            i,j=neighbor['select_x'],neighbor['select_y']
            vector=(i-x,j-y) #vector for (x,y) point to (i,j) neighbor
            angle=np.arctan2(vector[1],vector[0])
            angle_deg_list+=[(angle*180/np.pi)%(60)] #account for 6-fold symmetry
            psi6_list+=[np.exp(complex(0,6*angle))] #in rad. psi6 just converts angle to number from 0 to 1, accounting for 6-fold symmetry
        data.loc[particle_idx,'theta6']=circmean(angle_deg_list,high=60,low=0) #circular mean
        data.loc[particle_idx,'psi6']=abs(np.mean(psi6_list))
    return data
def find_defect_density(data,bin_parameter,params_dict,psi6cutoff=0.7): #doesn't work on cylinders bc pure cylinders can't be parametrized by r (distance from vertex)
    if bin_parameter=='r': #it makes sense to discretize bins based on r=a0=1, in this case binwidth=1
        bins=np.arange(0,np.max(abs(data['r'])),1)
    if bin_parameter=='circ': #based on r binning
        if params_dict['cthetadeg']!=0:
            bins=np.arange(0,np.max(abs(data['r'])),1)*params_dict['cthetadeg']*np.pi/180
        if params_dict['cthetadeg']==0:
            bins=np.arange(0,np.max(abs(data['circ'])),1)
    data_count,bin_edges=np.histogram(abs(data[bin_parameter]), bins=bins)
    defect_count,bin_edges=np.histogram(abs(data[data['psi6']<psi6cutoff][bin_parameter]), bins=bins)
    data_count[np.where(data_count==0)]=-1 #gets rid of division by 0 errors, since in defect_density 0/-1=0
    defect_density=defect_count/data_count
    return defect_count,data_count,defect_density,bin_edges,psi6cutoff
def plot_defect_density(defect_density,bin_edges,bin_parameter,title,ax,psi6cutoff):
    midpoint=(bin_edges[1]-bin_edges[0])/2 #since bins are from r1 to r2 or c1 to c2; find r0 or c0, which is the midline of the frustum
    ax.bar(bin_edges[:-1]+midpoint, defect_density, width = 1)
    ax.set_xlim(min(bin_edges), max(bin_edges))
    ax.set_ylim(0,1.05)
    ax.set_xlabel(bin_parameter)
    ax.set_ylabel('Defect density ('+r'$\psi_6$'+'<'+str(psi6cutoff)+')')
    ax.set_title(title+'\n')
def plot_defect_count(defect_count,bin_edges,bin_parameter,title,ax,psi6cutoff):
    midpoint=(bin_edges[1]-bin_edges[0])/2 #since bins are from r1 to r2 or c1 to c2; find r0 or c0, which is the midline of the frustum    
    ax.bar(bin_edges[:-1]+midpoint, defect_count, width = 1)
    ax.set_xlim(min(bin_edges), max(bin_edges))
    ax.set_ylim(0,10)
    ax.set_xlabel(bin_parameter)
    ax.set_ylabel('Defect count ('+r'$\psi_6$'+'<'+str(psi6cutoff)+')')
    ax.set_title(title+'\n')
def plot_gaussiankde(data,bin_parameter,ax,psi6cutoff=0.7): #plot_defect_density and plot_defect_count are manual binning histograms. the pandas implementation of gaussian KDE is an automatic method
    data[data['psi6']<psi6cutoff][bin_parameter].plot.kde(ax=ax,label=r'$\psi_6$'+'<'+str(psi6cutoff),bw_method=0.1)
def psi6cutoff_kde(data,psi6cutoff_list=np.linspace(0.5,0.9,5)):    
    figKDE=plt.figure(figsize=(10,5),dpi=400)
    axKDE_r=figKDE.add_subplot(121)
    axKDE_r.set_xlabel('r')
    axKDE_r.set_title('KDE of defects by '+r'$\psi_6$'+' cutoff, binned by r')
    
    axKDE_circ=figKDE.add_subplot(122)    
    axKDE_circ.set_xlabel('circ')
    axKDE_cir
    c.set_title('KDE of defects by '+r'$\psi_6$'+' cutoff, binned by circ')

    for psi6cutoff in psi6cutoff_list:
        plot_gaussiankde(data,'r',axKDE_r,psi6cutoff)
        plot_gaussiankde(data,'circ',axKDE_circ,psi6cutoff) 
    axKDE_r.legend()
    axKDE_circ.legend()

In [ ]:
def get_data_collect(run_dir_df_filtered):
    start=start_timer()
    data_collect_list=[]
    #####
    start=time.time()
    counter_total=len(run_dir_df_filtered['run_dir'].to_list())
    counter_current=0
    looptime_list=[]
    #####   
    for run_dir in run_dir_df_filtered['run_dir'].to_list():     
        ######
        loopstart=time.time()
        counter_current+=1
        ######  
        params_dict=get_params_dict(run_dir)              
        data,adj=load_data(run_dir,shifted=False) #TURN OFF DATA_SHIFTED CALCULATION (visualization purposes only but very slow)      
        data=find_images(data,params_dict)
        data=get_theta6_psi6(data,adj,run_dir)     
        for param in list(params_dict.items()): #attach params to data so that in data_collect we can find the individial trial data still
            data[param[0]]=param[1]
        data_collect_list+=[data] #make a list of dataframes, then append at the very end 
        ######
        loopend=time.time()
        looptime_list+=[loopend-loopstart]
        avglooptime=np.mean(looptime_list)
        if counter_current%5==1: #print time estimate every 5 loops
            print('time remaining: '+str(np.round(avglooptime*(counter_total-counter_current)/60,2))+' min',end='...')
        ######
    data_collect=pd.concat(data_collect_list,ignore_index=True)
    print('get_data_collect took '+str(np.round((time.time()-start)/60,2))+' min')
    return data_collect
def get_defect_density_collect(data_collect,psi6cutoff_list):
    defect_density_collect_list=[]
    defect_count_collect_list=[]
    data_count_collect_list=[]
    counter=0
    maxcounter=len(np.unique(data_collect['cthetadeg']))
    for cthetadeg in np.unique(data_collect['cthetadeg']):
        counter+=1
        print(counter,'/',maxcounter)
        for cinit in np.unique(data_collect['cinit']):
            for ithetadeg in np.unique(data_collect['ithetadeg']):
                for psi6cutoff in psi6cutoff_list:
                    for bin_parameter in ['r','circ']:
                        data_collect_filtered=data_collect[(data_collect['cthetadeg']==cthetadeg)&(data_collect['cinit']==cinit)&(data_collect['ithetadeg']==ithetadeg)] #     &(run_dir_df['trial']==0)
                        run_dir=data_collect_filtered['run_dir'].iloc[0] #just to get cone parameters
                        params_dict=get_params_dict(run_dir)
                        defect_count,data_count,defect_density,bin_edges,psi6cutoff=find_defect_density(data_collect_filtered,bin_parameter,params_dict,psi6cutoff=psi6cutoff)

                        defect_density=np.append(defect_density,0/(-1)) #since bin_edges goes from [0,max) where the bins are [0,1), [1,2)... so the max value (last value) bin should be 0. (-) is my way of notating that it's a missing particle instead of a true 0 defect density.
                        defect_density=np.reshape(defect_density,(1,len(defect_density)))            
                        defect_density=pd.DataFrame(defect_density,columns=bin_edges)

                        defect_count=np.append(defect_count,0/(-1))
                        defect_count=np.reshape(defect_count,(1,len(defect_count)))            
                        defect_count=pd.DataFrame(defect_count,columns=bin_edges)

                        data_count=np.append(data_count,0/(-1))
                        data_count=np.reshape(data_count,(1,len(data_count)))            
                        data_count=pd.DataFrame(data_count,columns=bin_edges)    
                        
                        params=[bin_parameter,cthetadeg,cinit,ithetadeg,psi6cutoff]
                        params=np.reshape(params,(1,len(params))) #note this changes everything to string type
                        params=pd.DataFrame(params,columns=['bin_parameter','cthetadeg','cinit','ithetadeg','psi6cutoff']) #dummy dataframe - it has correct value but incorrect type. single rows are annoying to work with so following lines just replace the value with correct type
                        params['cthetadeg']=cthetadeg #reassign with correct float type
                        params['cinit']=cinit
                        params['ithetadeg']=ithetadeg
                        params['psi6cutoff']=psi6cutoff
                        defect_density=pd.concat([params, defect_density], axis=1)
                        defect_count=pd.concat([params, defect_count], axis=1)   
                        data_count=pd.concat([params, data_count], axis=1)  
                        
                        defect_density_collect_list+=[defect_density]
                        defect_count_collect_list+=[defect_count]
                        data_count_collect_list+=[data_count]
    defect_density_collect=pd.concat(defect_density_collect_list,ignore_index=True)
    defect_count_collect=pd.concat(defect_count_collect_list,ignore_index=True)
    data_count_collect=pd.concat(data_count_collect_list,ignore_index=True)
        
    return defect_density_collect,defect_count_collect,data_count_collect
def defect_density_collect_plot(ax,defect_density_collect,fixed1key,fixed1val,fixed2key,fixed2val,fixed3key,fixed3val,legendvarkey,bin_parameter):
    for legendvarval in np.unique(defect_density_collect[legendvarkey]): #ithetadeg, for example
        title='defect density vs '+bin_parameter+',vary '+legendvarkey+'\n at '+str(fixed1key)+'='+str(fixed1val)+', '+str(fixed2key)+'='+str(fixed2val)+', '+str(fixed3key)+'='+str(fixed3val)+'\n'

        defect_density_collect_filtered=defect_density_collect[
        (defect_density_collect['bin_parameter']==bin_parameter)
        &(defect_density_collect[fixed3key]==fixed3val)
        &(defect_density_collect[fixed1key]==fixed1val)
        &(defect_density_collect[legendvarkey]==legendvarval)
        &(defect_density_collect[fixed2key]==fixed2val)].dropna(axis=1)

        defect_density=defect_density_collect_filtered.iloc[:,5:-1].values[0]
        bin_edges=defect_density_collect_filtered.iloc[:,5:].columns.values
        midpoint=(bin_edges[1]-bin_edges[0])/2 #since bins are from r1 to r2 or c1 to c2; find r0 or c0, which is the midline of the frustum
        ax.bar(bin_edges[:-1]+midpoint, defect_density, width = 1,alpha=0.1)
        ax.plot(bin_edges[:-1]+midpoint, defect_density,label=str(round(legendvarval,1)),alpha=1)        
        ax.set_xlim(min(bin_edges), max(bin_edges))
        ax.set_ylim(0,1.05)
        ax.set_xlabel(bin_parameter)
        ax.set_ylabel('Defect density')
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5),prop={'size': 6},title=legendvarkey)
        ax.set_title(title+'\n')
    return title
def collect_plot(ax,collectlabel,collect,fixed1key,fixed1val,fixed2key,fixed2val,fixed3key,fixed3val,legendvarkey,bin_parameter):
    for legendvarval in np.unique(collect[legendvarkey]): #ithetadeg, for example
        title=collectlabel+' vs '+bin_parameter+',vary '+legendvarkey+'\n at '+str(fixed1key)+'='+str(fixed1val)+', '+str(fixed2key)+'='+str(fixed2val)+', '+str(fixed3key)+'='+str(fixed3val)+'\n'

        #select a single parameter set, for all trials
        collect_filtered=collect[
        (collect['bin_parameter']==bin_parameter)
        &(collect[fixed3key]==fixed3val)
        &(collect[fixed1key]==fixed1val)
        &(collect[legendvarkey]==legendvarval)
        &(collect[fixed2key]==fixed2val)].dropna(axis=1)

        defect_density=collect_filtered.iloc[:,5:-1].values[0]
        bin_edges=collect_filtered.iloc[:,5:].columns.values
        midpoint=(bin_edges[1]-bin_edges[0])/2 #since bins are from r1 to r2 or c1 to c2; find r0 or c0, which is the midline of the frustum
        ax.plot(bin_edges[:-1]+midpoint, defect_density,label=str(round(legendvarval,1)),alpha=1)        
        ax.set_xlim(min(bin_edges), max(bin_edges))
        ax.set_ylim(0,1.05)
        ax.set_xlabel(bin_parameter)
        ax.set_title(title+'\n')
    return title
def collect_plot_sns(ax,collectlabel,collect,fixed1key,fixed1val,fixed2key,fixed2val,fixed3key,fixed3val,legendvarkey,bin_parameter,palette='deep',reverse_sizes=True):
    sns_df_list=[]
    if reverse_sizes:
        line_sizes=list(np.linspace(0.85,1.15,len(np.unique(collect[legendvarkey]))))[::-1]
    if not reverse_sizes:
        line_sizes=list(np.linspace(0.85,1.15,len(np.unique(collect[legendvarkey]))))
    for legendvarval in np.unique(collect[legendvarkey]): #ithetadeg, for example
        title=collectlabel+' vs '+bin_parameter+',vary '+legendvarkey+'\n at '+str(fixed1key)+'='+str(fixed1val)+', '+str(fixed2key)+'='+str(fixed2val)+', '+str(fixed3key)+'='+str(fixed3val)+'\n'

        #select a single parameter set, for all trials
        collect_filtered=collect[
            (collect['bin_parameter']==bin_parameter)
            &(collect[fixed3key]==fixed3val)
            &(collect[fixed1key]==fixed1val)
            &(collect[fixed2key]==fixed2val)
            &(collect[legendvarkey]==legendvarval)].dropna(axis=1)   

        defect_density=collect_filtered.iloc[:,5:-1].values[0]
        bin_edges=collect_filtered.iloc[:,5:].columns.values
        midpoint=(bin_edges[1]-bin_edges[0])/2 #since bins are from r1 to r2 or c1 to c2; find r0 or c0, which is the midline of the frustum

        sns_df=pd.DataFrame([])
        sns_df['x']=bin_edges[:-1]+midpoint
        sns_df['y']=defect_density
        sns_df['c']=legendvarval
        sns_df_list+=[sns_df]
    sns_df=pd.concat(sns_df_list,ignore_index=True)
    sns.lineplot(x=sns_df['x'], y=sns_df['y'],hue=sns_df['c'],palette=palette,ax=ax,ci=95,size=sns_df['c'],sizes=line_sizes)
    ax.set_xlim(min(bin_edges), max(bin_edges))
    ax.set_ylim(0,1.05)
    ax.set_xlabel(bin_parameter)
    ax.set_title(title+'\n')
    return title,sns_df
def find_data_collect_psi6cutoff(data_collect,psi6cutoff_list): #data_collect except reevaluate for each psi6 cutoff
    data_collect_psi6cutoff=pd.DataFrame([])
    for psi6cutoff in psi6cutoff_list:
        filtered=data_collect[data_collect['psi6']<psi6cutoff].copy()
        filtered['psi6cutoff']=psi6cutoff
        data_collect_psi6cutoff=data_collect_psi6cutoff.append(filtered)
    return data_collect_psi6cutoff
def data_collect_kde(fig,ax,data_collect_psi6cutoff,fixed1key,fixed1val,fixed2key,fixed2val,fixed3key,fixed3val,legendvarkey,bin_parameter):
    for legendvarval in np.unique(data_collect_psi6cutoff[legendvarkey]): #ithetadeg, for example
        title='defect kde vs '+bin_parameter+',vary '+legendvarkey+'\n at '+str(fixed1key)+'='+str(fixed1val)+', '+str(fixed2key)+'='+str(fixed2val)+', '+str(fixed3key)+'='+str(fixed3val)+'\n'

        #select a single parameter set, for all trials
        data_collect_psi6cutoff_filtered=data_collect_psi6cutoff[
        (data_collect_psi6cutoff[fixed3key]==fixed3val)
        &(data_collect_psi6cutoff[fixed1key]==fixed1val)
        &(data_collect_psi6cutoff[legendvarkey]==legendvarval)
        &(data_collect_psi6cutoff[fixed2key]==fixed2val)].dropna(axis=1)

        data_collect_psi6cutoff_filtered[bin_parameter].plot.kde(ax=ax,label=legendvarval)
        ax.set_xlabel(bin_parameter)
        ax.set_title(title+'\n')
    return title

In [ ]:
def export_data_collect():
    for jobnum in np.arange(1,100+1,1):
        run_dir_df_filtered=run_dir_df[((run_dir_df['trial']>=((jobnum-1)*1))&(run_dir_df['trial']<(jobnum*1)))]
        data_collect=get_data_collect(run_dir_df_filtered)
        info_tag=run_dir_df_info(run_dir_df_filtered)
        data_collect.to_pickle('data_collect_'+info_tag+'.pkl')
def compile_data_collect():
    start=time.time()
    counter=0
    pkl_file_list=[]
    for pkl_file in os.listdir(os.getcwd()): #get directory names in analysis folder
        if pkl_file.startswith('data_collect') and pkl_file.endswith('.pkl'):
            pkl_file_list+=[pkl_file]
            counter+=1
            print('.',end='')
    print('\n\n',counter,'files imported... loading data_collect...')
    data_collect_list=[]
    for pkl_file in pkl_file_list:
        with open(pkl_file, "rb") as fh: #read binary = rb, #file handle = fh
            data_collect = pickle.load(fh)
            data_collect_list += [data_collect]
            print('.',end='')
    data_collect=pd.concat(data_collect_list,ignore_index=True)
    print('done!',round((time.time()-start)/60,2),'min import')
    print('exporting data_collect_compiled.pkl...')
    start=time.time()
    data_collect.to_pickle('data_collect_compiled.pkl')
    print('done!',round((time.time()-start)/60,2),'min import')
def data_to_defect_collect():
    print('opening data_collect_compiled.pkl...')
    start=time.time()
    with open('data_collect_compiled.pkl', "rb") as fh:
        data_collect_compiled=pickle.load(fh)
    print('done!',round((time.time()-start)/60,2),'min import')

    data_collect=data_collect_compiled

    print('finding defect_density')
    psi6cutoff_list=[0.5,0.6,0.7,0.8,0.9] #psi6cutoff for defining a defect
    data_collect_psi6cutoff=find_data_collect_psi6cutoff(data_collect,psi6cutoff_list) #used for finding kde... kind of redundant with the defect_density code but whatever i don't want to rewrite it right now
    defect_density_collect,defect_count_collect,data_count_collect=get_defect_density_collect(data_collect,psi6cutoff_list)
    print('data_collect pkl input:\n','data_collect_compiled.pkl')
    print('\n imported data_collect pickle file. \n found data_collect_psi6cutoff, \n defect_density_collect, defect_count_collect, data_count_collect for',len(np.unique(data_collect['trial'])),'trials at each parameter set \n\n')
    defect_density_collect_dict={'trials':len(np.unique(data_collect['trial']))}
    for item in defect_density_collect.columns[:5]:
        print(item,'\n',np.unique(defect_density_collect[item]))
        defect_density_collect_dict.update({item:np.unique(defect_density_collect[item])})

    print('\ndefect_density_collect_dict\n',defect_density_collect_dict,'\n')

    defect_density_collect.head()

    print('exporting defect_density_collect.pkl...')
    start=time.time()
    defect_density_collect.to_pickle('defect_density_collect.pkl')
    print('done!',round((time.time()-start)/60,2),'min export')
def import_collect_pkl():
    print('opening defect_density_collect.pkl...')
    start=time.time()
    with open(output_dir+'/defect_density_collect.pkl', "rb") as fh:
        defect_density_collect=pickle.load(fh)
    print('done!',round((time.time()-start)/60,2),'min import')
    return defect_density_collect

if os.path.exists('defect_density_collect.pkl.zip'): #intermediate analysis file of dataset
    defect_density_collect=unzip_input('defect_density_collect.pkl.zip')
    print('imported defect_density_collect.pkl.zip as defect_density_collect')
elif os.path.exists('defect_density_collect.pkl'): #intermediate analysis file of dataset
    defect_density_collect=unzip_input('defect_density_collect.pkl')
    print('imported defect_density_collect.pkl as defect_density_collect')    
else: #warning: the following steps will take more than a few days on a cluster to run.
    export_data_collect() #generates and exports data_collect for each trial number 
    compile_data_collect() #generates and exports a large, compiled data_collect
    data_to_defect_collect() #converts data_collect to defect_density_collect, and exports
    defect_density_collect=import_collect_pkl() #reimport defect_density_collect
    print('generated and imported defect_density_collect.pkl as defect_density_collect')

In [ ]:
def export_fig7cylinder_data():
    fig7cylinder_data_list=[]
    counter=0
    start=time.time()

    cthetadeg=0
    ithetadeg_list=[0,10,20,30,40,50,60]
    trials_list=np.unique(run_dir_df['trial'])
    cinit_list=np.unique(run_dir_df['cinit'])
    psi6cutoff_list=[0.9]
    for psi6cutoff in psi6cutoff_list:
        print('psi6cutoff ',psi6cutoff,end='...')
        for ithetadeg in ithetadeg_list:
            print('ithetadeg ',ithetadeg,end='...')

            fig7cylinder_data_list=[]
            for cinit in cinit_list:
                counter+=1
                print('cinit ',counter,'/',len(cinit_list),end='...')
                for trial in trials_list:
                    print('trial ',trial,end='...')

                    run_dir_df_filtered=run_dir_df[
                        (run_dir_df['cthetadeg']==cthetadeg)
                        &(run_dir_df['cinit']==cinit)
                        &(run_dir_df['ithetadeg']==ithetadeg)
                        &(run_dir_df['trial']==trial)]

                    run_dir=run_dir_df_filtered['run_dir'][0]
                    params_dict=get_params_dict(run_dir)
                    data,adj=load_data(run_dir)
                    data=find_images(data,params_dict)
                    data=get_theta6_psi6(data,adj,run_dir)

                    Ndefect=len(data[data['psi6']<psi6cutoff])
                    Nactual=len(data['psi6'])
                    defectdensity=Ndefect/Nactual

                    fig7cylinder_dict={'cthetadeg':cthetadeg,'cinit':cinit,'ithetadeg':ithetadeg,'trial':trial,'avg_psi6':np.mean(data['psi6']),
                               'psi6cutoff':psi6cutoff,'Ndefect':Ndefect,'Nactual':Nactual,'defectdensity':defectdensity}
                    fig7cylinder_data=pd.DataFrame([fig7cylinder_dict])
                    fig7cylinder_data_list+=[fig7cylinder_data]    
            fig7cylinder_data=pd.concat(fig7cylinder_data_list)
            print('done!',round((time.time()-start)/60,2),'min to process')

            print('exporting fig7cylinder_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'.pkl...')
            start=time.time()
            fig7cylinder_data.to_pickle('fig7cylinder_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'.pkl')
            print('done!',round((time.time()-start)/60,2),'min export')
def compile_fig7_data():
    fig7cylinder_data_list=[] #find defect density of cylinders. this is because my existing find_defect_density function doesn't work for cylinders (it assumed everything was able to be parametrized by r back then)
    start=time.time()

    cthetadeg=0
    ithetadeg_list=[0,10,20,30,40,50,60]
    psi6cutoff_list=[0.9]

    start=time.time()
    for psi6cutoff in psi6cutoff_list:
        for ithetadeg in ithetadeg_list:
            if os.path.exists('fig7cylinder_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'.pkl'):
                print('opening','fig7cylinder_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'.pkl','...')
                with open('fig7cylinder_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'.pkl', "rb") as fh:
                    fig7cylinder_data_list+=[pickle.load(fh)]
            if os.path.exists('fig7cylinder_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'_cinit0-5'+'.pkl'):
                print('opening','fig7cylinder_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'_cinit0-5'+'.pkl','...')
                with open('fig7cylinder_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'_cinit0-5'+'.pkl', "rb") as fh:
                    fig7cylinder_data_list+=[pickle.load(fh)]                    
    fig7cylinder_data=pd.concat(fig7cylinder_data_list)
    print('done!',round((time.time()-start)/60,2),'min import')

    print('exporting fig7cylinder_data.pkl...')
    start=time.time()
    fig7cylinder_data.to_pickle('fig7cylinder_data.pkl')
    print('done!',round((time.time()-start)/60,2),'min export')
def import_fig7cylinder_data():
    print('opening fig7cylinder_data.pkl...')
    start=time.time()
    with open('fig7cylinder_data.pkl', "rb") as fh:
        fig7cylinder_data=pickle.load(fh)
    print('done!',round((time.time()-start)/60,2),'min import')        
    return fig7cylinder_data

if not os.path.exists('fig7cylinder_data.pkl'): #aggregated fig7cylinder.pkl
    if not os.path.exists('fig7cylinder_data_ithetadeg-40_psi6cutoff-0.9.pkl'): #individual fig7cylinder files for a set of cinit
        export_fig7cylinder_data() #might take a few days to run while on cluster        
    compile_fig7_data() #compile fig7cylinder_data into fig7_data

fig7cylinder_data=import_fig7cylinder_data()

In [ ]:
cone_cylinder_df=pd.DataFrame([]) #combine defect density info from defect_density_collect (which has cone defect info) with fig7cylinder_data (cylinder defect info)

cone_df_list=[]
start=time.time()
collect=defect_density_collect[(defect_density_collect['bin_parameter']=='circ')&(defect_density_collect['cthetadeg']>0)&(defect_density_collect['cthetadeg']<=30)]
for cthetadeg in np.unique(collect['cthetadeg']):
    for ithetadeg in np.unique(collect['ithetadeg']):
        for cinit in np.unique(collect['cinit']):
            for psi6cutoff in np.unique(collect['psi6cutoff']):
                collect_filtered=collect[(collect['cthetadeg']==cthetadeg)&(collect['ithetadeg']==ithetadeg)&(collect['cinit']==cinit)&(collect['psi6cutoff']==psi6cutoff)].dropna(axis=1)
                defect_density=collect_filtered.iloc[:,5:-1].values[0]
                bin_edges=collect_filtered.iloc[:,5:].columns.values
                midpoint=(bin_edges[1]-bin_edges[0])/2 #since bins are from r1 to r2 or c1 to c2; find r0 or c0, which is the midline of the frustum
                cone_df=pd.DataFrame([])                
                cone_df['circ']=bin_edges[:-1]+midpoint
                cone_df['defect_density']=defect_density
                cone_df['cthetadeg']=cthetadeg
                cone_df['ithetadeg']=ithetadeg
                cone_df['cinit']=cinit
                cone_df['psi6cutoff']=psi6cutoff
                cone_df_list+=[cone_df]
cone_df=pd.concat(cone_df_list)

cylinder_df_list=[]
mean_cylinder_df_list=[]
cylinder_df=fig7cylinder_data
for cthetadeg in np.unique(cylinder_df['cthetadeg']):            
    for ithetadeg in np.unique(cylinder_df['ithetadeg']):            
        for cinit in np.unique(cylinder_df['cinit']):                        
            for psi6cutoff in np.unique(cylinder_df['psi6cutoff']):                                        
                mean_defect_density=np.mean(cylinder_df[(cylinder_df['cthetadeg']==cthetadeg)&(cylinder_df['ithetadeg']==ithetadeg)
                &(cylinder_df['cinit']==cinit)&(cylinder_df['psi6cutoff']==psi6cutoff)]['defectdensity'])
                mean_cylinder_df=pd.DataFrame([{'defect_density':mean_defect_density,'cthetadeg':cthetadeg,'ithetadeg':ithetadeg,'cinit':cinit,'psi6cutoff':psi6cutoff}])
                mean_cylinder_df_list+=[mean_cylinder_df]
mean_cylinder_df=pd.concat(mean_cylinder_df_list)
for cinit in np.unique(collect['cinit']): #repeat cylinder values for all cinit (so that we can plot it with the cone values) - this means that cinit value won't really be physical for cylinders!!!
    cylinder_df=pd.DataFrame([])
    cylinder_df['circ']=mean_cylinder_df['cinit'] #CYLINDER cinit.
    cylinder_df['defect_density']=mean_cylinder_df['defect_density']
    cylinder_df['cthetadeg']=mean_cylinder_df['cthetadeg']
    cylinder_df['ithetadeg']=mean_cylinder_df['ithetadeg']
    cylinder_df['cinit']=cinit #CONE cinit. not the actual cylinder cinit.
    cylinder_df['psi6cutoff']=mean_cylinder_df['psi6cutoff']
    cylinder_df_list+=[cylinder_df]
cylinder_df=pd.concat(cylinder_df_list)

cone_cylinder_df=pd.concat([cone_df,cylinder_df])
print(round((time.time()-start)/60,2),'min to process')

In [ ]:
def find_coeff_df(cone_cylinder_df,xcutoff_offset,xcutoff_max,ithetadeg_list,cinit_list,psi6cutoff_list): #find coefficients for best exponential fit
    coeff_df_list=[]
    for ithetadeg in ithetadeg_list:
        for cinit in cinit_list:
            for psi6cutoff in psi6cutoff_list:
                cone_cylinder_df_filtered=cone_cylinder_df[(cone_cylinder_df['ithetadeg']==ithetadeg)&(cone_cylinder_df['psi6cutoff']==psi6cutoff)&(cone_cylinder_df['cinit']==cinit)]
                df_sns=pd.DataFrame([])
                df_sns['x']=cone_cylinder_df_filtered['circ']
                df_sns['y']=cone_cylinder_df_filtered['defect_density']
                df_sns['c']=cone_cylinder_df_filtered['cthetadeg']

                xcutoff_offset=xcutoff_offset
                xcutoff_max=xcutoff_max
                for cvalue in np.unique(df_sns['c']): #find fits for each dataset

                    df_sns_filtered=df_sns[(df_sns['c']==cvalue)&(df_sns['y']>=0.00001)] #select dataset ('c'). y needs to be nonzero.

                    ymax=max(df_sns_filtered['y']) #find x cutoffs
                    xcutoff_left=df_sns_filtered[df_sns_filtered['y']==ymax]['x'].to_numpy()[0]+xcutoff_offset
                    df_sns_filtered=df_sns_filtered[df_sns_filtered['x']>=xcutoff_left]

                    ymin=min(df_sns_filtered['y'])
                    xcutoff_right=df_sns_filtered[df_sns_filtered['y']==ymin]['x'].to_numpy()[0]
                    if xcutoff_max[0]==True:
                        xcutoff_right=xcutoff_max[1]
                    df_sns_filtered=df_sns_filtered[df_sns_filtered['x']<=xcutoff_right]

                    x_data=df_sns_filtered['x'].astype(float)
                    y_data=df_sns_filtered['y'].astype(float)

                    log_y_data = np.log(y_data)
                    a,b = np.polyfit(x_data, log_y_data, 1) #fit line to log(y) vs. x data, gives straight line if exponential relationship between y and x
                    y=np.exp(a*x_data+b)
                    
                    coeff_df=pd.DataFrame([])
                    coeff_df['circ_filtered']=x_data
                    coeff_df['defect_density']=y_data
                    coeff_df['defect_density_fit']=y
                    coeff_df['a']=a #y = exp(a*x+b)
                    coeff_df['b']=b #y = exp(a*x+b)
                    coeff_df['psi6cutoff']=psi6cutoff
                    coeff_df['ithetadeg']=ithetadeg
                    coeff_df['cthetadeg']=cvalue
                    coeff_df['cinit']=cinit            
                    coeff_df_list+=[coeff_df]
    coeff_df=pd.concat(coeff_df_list) 
    return coeff_df

cinit_list=[10]

psi6cutoff_list=[0.9]
ithetadeg_list=np.unique(cone_cylinder_df[cone_cylinder_df['cthetadeg']==0]['ithetadeg'])
xcutoff_offset=0 #offset calculated left cutoff value
xcutoff_max=[True,10] #right cutoff at fixed value
coeff_df=find_coeff_df(cone_cylinder_df,xcutoff_offset,xcutoff_max,ithetadeg_list,cinit_list,psi6cutoff_list) #find coefficients for best exponential fit
cthetadeg_list=np.unique(coeff_df['cthetadeg'])
for cthetadeg in cthetadeg_list:
    print('cthetadeg: '+str(cthetadeg),'| left cutoff: '+str(round(min(coeff_df[coeff_df['cthetadeg']==cthetadeg]['circ_filtered']),2)),
          '| right cutoff: '+str(round(max(coeff_df[coeff_df['cthetadeg']==cthetadeg]['circ_filtered']),2)))

In [ ]:
axis_color='0.3'
n_cone=6 #5,10,15...30
n_cylinder=11 #2,3,4,...,12
cylinder_palette=sns.color_palette(c_palette[:n_cylinder][6:7]) #select C=10
cone_palette=sns.color_palette(phi_palette[2:2+n_cone])

psi6cutoff=0.9
ithetadeg=40
cinit=10
cone_cylinder_df_filtered=cone_cylinder_df[(cone_cylinder_df['psi6cutoff']==psi6cutoff)&(cone_cylinder_df['ithetadeg']==ithetadeg)&(cone_cylinder_df['cinit']==cinit)] #orig data

fig=plt.figure(figsize=(5,2.5),dpi=400)
ax=fig.add_subplot(111)
cone=cone_cylinder_df_filtered[cone_cylinder_df_filtered['cthetadeg']!=0]
cone=cone.reset_index()
cylinder=cone_cylinder_df_filtered[cone_cylinder_df_filtered['cthetadeg']==0]

line_sizes=list(np.linspace(0.85,1.15,n_cone))
sns.scatterplot(x=cylinder['circ'], y=cylinder['defect_density'],color=cylinder_palette[0],ax=ax,legend=False,zorder=1) 
sns.lineplot(x=cone['circ'], y=cone['defect_density'],hue=cone['cthetadeg'],palette=cone_palette,ax=ax,legend=True,zorder=0,size=cone['cthetadeg'],sizes=line_sizes)    

handles,labels = ax.get_legend_handles_labels()
new_labels_list=[]
new_labels_list+=[r'$\phi$='+str(0)+r'$^{\circ}$'] #cylinder label
handles+=[handles[0]] #add an additional entry for cylinder
for label in labels:
    new_labels_list+=[r'$\phi$='+str(int(float(label)))+r'$^{\circ}$'] #cone labels
new_labels = new_labels_list
ax.legend(handles,new_labels,labelcolor=cylinder_palette+cone_palette,loc='upper right',frameon=False,handlelength=0,handletextpad=0,labelspacing=0)

xlimL,xlimR=0,12
ax.set_xlim(xlimL,xlimR)
ax.set_ylim(0,1)

title='defect density'+' vs '+'circ'+', ithetadeg='+str(ithetadeg)+'\npsi6cutoff='+str(0.9)+', cinit='+str(cinit)+'\n'
ax.set_title(title+'\n')
ax.set_ylabel(r'$\rho$');
ax.set_xlabel(r'$C$');        
ax.annotate('increasing '+r'$\phi$',xy=(5, 0.1), xycoords='data',xytext=(6, 0.5), textcoords='data',arrowprops=dict(arrowstyle="->",connectionstyle="arc3,rad=-0.2",color=axis_color),color=axis_color)

ax.tick_params(direction="in",colors=axis_color)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_color(axis_color)
ax.spines['left'].set_color(axis_color)
ax.xaxis.label.set_color(axis_color)
ax.yaxis.label.set_color(axis_color)

plt.savefig('analysis/cylinderplot.svg',bbox_inches = "tight")
plt.show()

In [ ]:
fig=plt.figure(figsize=(5,2.5),dpi=400)
ax=fig.add_subplot(111)

line_sizes=list(np.linspace(0.85,1.15,n_cone))
sns.scatterplot(x=cylinder['circ'], y=cylinder['defect_density'],color=cylinder_palette[0],ax=ax,legend=False,zorder=1) 
sns.lineplot(x=cone['circ'], y=cone['defect_density'],hue=cone['cthetadeg'],palette=cone_palette,ax=ax,legend=True,zorder=0,size=cone['cthetadeg'],sizes=line_sizes)    

ax.set_yscale('log')
ax.set_ylabel(r"$log(\rho)$")
ax.set_xlabel(r'$C$')

xlimL,xlimR=0,12
ax.set_xlim(xlimL,xlimR)
ax.set_ylim(0,1.05)

title='defect density'+' vs '+'circ'+', ithetadeg='+str(ithetadeg)+'\npsi6cutoff='+str(0.9)+', cinit='+str(cinit)+'\n'
ax.set_title(title+'\n')
ax.set_ylabel(r'$\rho$');
ax.set_xlabel(r'$C$');        
ax.annotate('increasing '+r'$\phi$',xy=(5, 0.1), xycoords='data',xytext=(6, 0.5), textcoords='data',arrowprops=dict(arrowstyle="->",connectionstyle="arc3,rad=-0.2",color=axis_color),color=axis_color)

ax.tick_params(which="both",direction="in",colors=axis_color)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_color(axis_color)
ax.spines['left'].set_color(axis_color)
ax.xaxis.label.set_color(axis_color)
ax.yaxis.label.set_color(axis_color)

handles,labels = ax.get_legend_handles_labels()
new_labels_list=[]
new_labels_list+=[r'$\phi$='+str(0)+r'$^{\circ}$'] #cylinder label
handles+=[handles[0]] #add an additional entry for cylinder
for label in labels:
    new_labels_list+=[r'$\phi$='+str(int(float(label)))+r'$^{\circ}$'] #cone labels
new_labels = new_labels_list
ax.legend(handles,new_labels,labelcolor=cylinder_palette+cone_palette,loc='upper right',frameon=False,handlelength=0,handletextpad=0,labelspacing=0)

plt.savefig('cylinderplot_log.svg',bbox_inches = "tight")
plt.show()

In [ ]:
palette=cylinder_palette+cone_palette
coeff_df_filtered=coeff_df[(coeff_df['psi6cutoff']==psi6cutoff)&(coeff_df['ithetadeg']==ithetadeg)&(coeff_df['cinit']==cinit)].reset_index()
cone_cylinder_df_filtered=cone_cylinder_df[(cone_cylinder_df['psi6cutoff']==psi6cutoff)&(cone_cylinder_df['ithetadeg']==ithetadeg)&(cone_cylinder_df['cinit']==cinit)]

fig=plt.figure(figsize=(5,2.5),dpi=400)
ax=fig.add_subplot(111)
line_sizes=[1]+list(np.linspace(0.85,1.15,n_cone)) #first line is for cylinder. rest is for cones
sns.lineplot(x=coeff_df_filtered['circ_filtered'], y=coeff_df_filtered['defect_density_fit'],hue=coeff_df_filtered['cthetadeg'],palette=palette,ax=ax,legend=False,size=coeff_df_filtered['cthetadeg'],sizes=line_sizes) #line of best fit
sns.scatterplot(x=coeff_df_filtered['circ_filtered'], y=coeff_df_filtered['defect_density'],hue=coeff_df_filtered['cthetadeg'],alpha=0.25,s=5,palette=palette,ax=ax,legend=False)
sns.scatterplot(x=cone_cylinder_df_filtered['circ'], y=cone_cylinder_df_filtered['defect_density'],hue=cone_cylinder_df_filtered['cthetadeg'],alpha=0.25,palette=palette,s=5,ax=ax,legend=False)    

ax.set_yscale('log')
ax.set_ylabel(r"$log(\rho)$")
ax.set_xlabel(r'$C$')

xlimL,xlimR=0,12
ax.set_xlim(xlimL,xlimR)
ax.set_ylim(0,1.05)

handles,labels = ax.get_legend_handles_labels()
new_labels_list=[]
for label in labels:
    new_labels_list+=[r'$\phi$='+str(label)+r'$^{\circ}$']
new_labels = new_labels_list
ax.annotate('increasing '+r'$\phi$',xy=(5, 0.1), xycoords='data',xytext=(6, 0.5), textcoords='data',arrowprops=dict(arrowstyle="->",connectionstyle="arc3,rad=-0.2",color=axis_color),color=axis_color)

ax.tick_params(which="both",direction="in",colors=axis_color)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_color(axis_color)
ax.spines['left'].set_color(axis_color)
ax.xaxis.label.set_color(axis_color)
ax.yaxis.label.set_color(axis_color)

title='defect density'+' vs '+'circ'+', ithetadeg='+str(ithetadeg)+'\npsi6cutoff='+str(0.9)+', cinit='+str(cinit)+'\n'
ax.set_title(title+'\n')
ax.set_ylabel(r'$\rho$');
ax.set_xlabel(r'$C$');        

plt.savefig('cylinderplot_logcoeff.svg',bbox_inches = "tight")
plt.show()

In [ ]:
cthetadeg=0
ithetadeg=40
trial=0

cinit_list=[2,4,6,8,10,12]

ref_ylim_range=reference_range+50+10 #this is the standard range used for all of my plots so far
new_ylim_range=reference_range+5

ylim_range_ratio=new_ylim_range/ref_ylim_range

for cinit in cinit_list:
    run_dir_df_filtered=run_dir_df[
        (run_dir_df['cthetadeg']==cthetadeg)
        &(run_dir_df['cinit']==cinit)
        &(run_dir_df['ithetadeg']==ithetadeg)
        &(run_dir_df['trial']==trial)]

    run_dir=run_dir_df_filtered['run_dir'][0]
    params_dict=get_params_dict(run_dir)
    data,adj=load_data(run_dir)
    data=find_images(data,params_dict)
    data=get_theta6_psi6(data,adj,run_dir)

    fig=plt.figure(figsize=(6.5,6.5*ylim_range_ratio),dpi=400)
    ax=fig.add_subplot(111)
    plot_sector(data,params_dict,ax,c='coordnum',cmap='newcmp')
    
    ylim_left=min(data['y'])-10
    ax.set_ylim(ylim_left,ylim_left+new_ylim_range) #the max is always y=20 above the nucleation seed. so we really only need to find the lower bound    
    ax.invert_yaxis() #setting ylim flips the axis..so we gotta flip it back..
    
    Nactual=len(data)
    info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)
    plt.savefig('cylinder_cinit'+str(cinit)+'.svg',bbox_inches = "tight")
    plt.show()

In [ ]:
cinit_list=[2,4,6,8,10,12]
fig=plt.figure(figsize=(0.75,2.5),dpi=400)
ax=fig.add_subplot(111)

cylinder_palette_sliced=sns.color_palette(
    c_palette[:n_cylinder][0:1]+ #C=2
    c_palette[:n_cylinder][2:3]+ #C=4
    c_palette[:n_cylinder][4:5]+ #C=6
    c_palette[:n_cylinder][6:7]+ #C=8
    c_palette[:n_cylinder][8:9]+ #C=10            
    c_palette[:n_cylinder][10:11] #C=12                
)
n=len(cinit_list)
palette=cylinder_palette_sliced
test_df=pd.DataFrame({'x':np.linspace(0,1,n),'y':np.linspace(0,1,n),'c':cinit_list})
sns.lineplot(x=test_df['x'],y=test_df['y'],hue=test_df['c'],ax=ax,palette=palette,legend=True)
ax.set_ylabel(r'$\rho$')
ax.set_xlabel(r'$C$')

handles,labels = ax.get_legend_handles_labels()
new_labels_list=[]
for label in labels:
    new_labels_list+=[r'$C$='+str(label)] #r'$^{\circ}$'
new_labels = new_labels_list
ax.legend(handles,new_labels,labelcolor='linecolor',loc='upper left',frameon=False,handlelength=0, handletextpad=0,labelspacing=1) #handlelength=0, handletextpad=0 removes the legend markers
ax.axis('off')
plt.savefig('analysis/cinitlegend.svg',bbox_inches = "tight")

In [ ]:
ithetadeg_list=[0,10,20,30]
fig=plt.figure(figsize=(0.75,2.5),dpi=400)
ax=fig.add_subplot(111)

n=len(ithetadeg_list)
# palette=sns.color_palette(sns.dark_palette(sns.color_palette("bright",as_cmap=True)[6],n+1,reverse=True)[1:n+1])
palette=theta_palette[2:2+n]
test_df=pd.DataFrame({'x':np.linspace(0,1,n),'y':np.linspace(0,1,n),'c':ithetadeg_list})
sns.lineplot(x=test_df['x'],y=test_df['y'],hue=test_df['c'],ax=ax,palette=palette,legend=True)
ax.set_ylabel(r'$\rho$')
ax.set_xlabel('C')
########
handles,labels = ax.get_legend_handles_labels()
new_labels_list=[]
for label in labels:
    new_labels_list+=[r'$\theta$='+str(label)+ r'$^{\circ}$'] #r'$^{\circ}$'
new_labels = new_labels_list
ax.legend(handles,new_labels,labelcolor='linecolor',loc='upper left',frameon=False,handlelength=0, handletextpad=0,labelspacing=1) #handlelength=0, handletextpad=0 removes the legend markers
ax.axis('off')

plt.savefig('thetalegend.svg',bbox_inches = "tight")

In [ ]:
axis_color='0.3'
defect_density_collect['ithetadeg']=np.array(defect_density_collect['ithetadeg']).astype(int) #because it bothers me that the orientation values have decimals lol

cthetadeg=5
cinit_list=[10]
ithetadeg=30
psi6cutoff=0.9

collect=defect_density_collect[defect_density_collect['ithetadeg']<=30]
n=len(np.unique(defect_density_collect[defect_density_collect['ithetadeg']<=30]['ithetadeg']))
collectlabel='defect density'

for cinit in cinit_list:
    fig=plt.figure(figsize=(5,2.5),dpi=400)
    ax=fig.add_subplot(111)
    fixed1key='cinit'
    fixed1val=cinit
    fixed2key='psi6cutoff'
    fixed2val=psi6cutoff
    fixed3key='cthetadeg'
    fixed3val=cthetadeg

    legendvarkey='ithetadeg'
    bin_parameter='r'
    
    title,sns_df=collect_plot_sns(ax,collectlabel,collect,fixed1key,fixed1val,fixed2key,fixed2val,fixed3key,fixed3val,legendvarkey,bin_parameter,palette,reverse_sizes=False)
    ax.set_ylabel(r'$\rho$')
    ax.set_xlabel(r'$R$')

    handles,labels = ax.get_legend_handles_labels()
    new_labels_list=[]
    for label in labels:
        new_labels_list+=[r'$\theta$='+str(label)+r'$^{\circ}$']
    new_labels = new_labels_list
    ax.legend(handles,new_labels,labelcolor='linecolor',loc='upper right',frameon=False,handlelength=0, handletextpad=0,labelspacing=0.2) #handlelength=0, handletextpad=0 removes the legend markers

    ax.set_xlim(0,max(plt.gca().lines[0].get_xdata()))
    ax.set_ylim(0,1)    
    
    ax.tick_params(direction="in",colors=axis_color)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['left'].set_color(axis_color)
    ax.xaxis.label.set_color(axis_color)
    ax.yaxis.label.set_color(axis_color)    
    Nactual=len(data)
    info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)+'_psi6cutoff-'+str(psi6cutoff)

    ax=plt.gca()
    line3=ax.lines[3]
    data3=line3.get_xydata()
    line2=ax.lines[2]
    data2=line2.get_xydata()    
    line1=ax.lines[1]
    data1=line1.get_xydata()        
    line0=ax.lines[0]
    data0=line0.get_xydata()            
plt.savefig('analysis/ithetadeg_plot.svg',bbox_inches = "tight")        

In [ ]:
cthetadeg=5
cinit_list=[10]

ithetadeg_list=[0,10,20,30]
trials_list=[0]
for cinit in cinit_list:    
    for trial in trials_list:
        for ithetadeg in ithetadeg_list:
            run_dir_df_filtered=run_dir_df[
                (run_dir_df['cthetadeg']==cthetadeg)
                &(run_dir_df['cinit']==cinit)
                &(run_dir_df['ithetadeg']==ithetadeg)
                &(run_dir_df['trial']==trial)]

            run_dir=run_dir_df_filtered['run_dir'][0]
            params_dict=get_params_dict(run_dir)
            data,adj=load_data(run_dir)
            data=find_images(data,params_dict)
            data=get_theta6_psi6(data,adj,run_dir)

            fig=plt.figure(figsize=(6.5,6.5),dpi=400)
            ax=fig.add_subplot(111)
            plot_sector(data,params_dict,ax,c='coordnum',cmap='newcmp')

            ax.set_ylim(min(data['y'])-10,min(data['y'])+reference_range+50)
            ax.invert_yaxis()

            Nactual=len(data)
            info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)

            plt.savefig('analysis/ithetadeg'+str(ithetadeg)+'.svg',bbox_inches = "tight")    
            plt.show()

In [ ]:
axis_color='0.3'
defect_density_collect['cthetadeg']=np.array(defect_density_collect['cthetadeg']).astype(int) #because it bothers me that the values have decimals lol

cthetadeg=5
cinit_list=[10]
ithetadeg=40
trial=0
psi6cutoff=0.9

collect=defect_density_collect[(defect_density_collect['cthetadeg']>=5)&(defect_density_collect['cthetadeg']<=30)]
collectlabel='defect density'

for cinit in cinit_list:
    fig=plt.figure(figsize=(5,2.5),dpi=400)
    ax=fig.add_subplot(111)
    fixed1key='cinit'
    fixed1val=cinit
    fixed2key='psi6cutoff'
    fixed2val=psi6cutoff
    fixed3key='ithetadeg'
    fixed3val=ithetadeg

    legendvarkey='cthetadeg'
    bin_parameter='r'

    n=len(np.unique(collect[legendvarkey])) #SKIP THE FIRST COLOR (BLUE) SINCE WE DON'T WANT TO PLOT THE CYLINDER (BLUE)
    palette=cone_palette
    title,df_sns=collect_plot_sns(ax,collectlabel,collect,fixed1key,fixed1val,fixed2key,fixed2val,fixed3key,fixed3val,legendvarkey,bin_parameter,palette,reverse_sizes=False)
    handles,labels = ax.get_legend_handles_labels()
    new_labels_list=[]
    for label in labels:
        new_labels_list+=[r'$\phi$='+str(label)+r'$^{\circ}$']
    new_labels = new_labels_list
    ax.legend(handles,new_labels,labelcolor='linecolor',loc='upper right',frameon=False,handlelength=0, handletextpad=0,labelspacing=0) #handlelength=0, handletextpad=0 removes the legend markers
    ax.set_ylabel(r'$\rho$')
    ax.set_xlabel(r'$R$')
    ax.set_xlim(0,max(plt.gca().lines[0].get_xdata()))
    ax.set_ylim(0,1) 
    ax.tick_params(direction="in",colors=axis_color)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['left'].set_color(axis_color)
    ax.xaxis.label.set_color(axis_color)
    ax.yaxis.label.set_color(axis_color)        
    plt.savefig('analysis/cthetadegR.svg',bbox_inches = "tight")

    fig=plt.figure(figsize=(5,2.5),dpi=400)
    ax=fig.add_subplot(111)
    bin_parameter='circ'
    title,df_sns=collect_plot_sns(ax,collectlabel,collect,fixed1key,fixed1val,fixed2key,fixed2val,fixed3key,fixed3val,legendvarkey,bin_parameter,palette,reverse_sizes=False)
    handles,labels = ax.get_legend_handles_labels()
    new_labels_list=[]
    for label in labels:
        new_labels_list+=[r'$\phi$='+str(label)+r'$^{\circ}$']
    new_labels = new_labels_list
    ax.legend(handles,new_labels,labelcolor='linecolor',loc='upper right',frameon=False,handlelength=0, handletextpad=0,labelspacing=0) #handlelength=0, handletextpad=0 removes the legend marker
    ax.set_xlabel(r'$C$')    
    ax.set_ylabel(r'$\rho$')    
    ax.set_xlim(0,12)
    ax.set_ylim(0,1) 
    
    ax.tick_params(direction="in",colors=axis_color)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['left'].set_color(axis_color)
    ax.xaxis.label.set_color(axis_color)
    ax.yaxis.label.set_color(axis_color)    
    
    Nactual=len(data)
    info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)+'_psi6cutoff-'+str(psi6cutoff)
    plt.savefig('analysis/cthetadegC.svg',bbox_inches = "tight")

In [ ]:
cthetadeg_list=[5,10,15,20,25,30]
fig=plt.figure(figsize=(0.75,2.5),dpi=400)
ax=fig.add_subplot(111)

palette=cone_palette
test_df=pd.DataFrame({'x':np.linspace(0,1,n),'y':np.linspace(0,1,n),'c':cthetadeg_list})
sns.lineplot(x=test_df['x'],y=test_df['y'],hue=test_df['c'],ax=ax,palette=palette,legend=True)
ax.set_ylabel(r'$\rho$')
ax.set_xlabel('C')

handles,labels = ax.get_legend_handles_labels()
new_labels_list=[]
for label in labels:
    new_labels_list+=[r'$\phi$='+str(label)+ r'$^{\circ}$'] #r'$^{\circ}$'
new_labels = new_labels_list
ax.legend(handles,new_labels,labelcolor='linecolor',loc='upper left',frameon=False,handlelength=0, handletextpad=0,labelspacing=1) #handlelength=0, handletextpad=0 removes the legend markers
ax.axis('off')
plt.savefig('analysis/cthetalegend.svg',bbox_inches = "tight")

In [ ]:
cthetadeg_list=[5,10,15]
cinit_list=[10]
ithetadeg=40
trial=0
for cthetadeg in cthetadeg_list:
    run_dir_df_filtered=run_dir_df[
        (run_dir_df['cthetadeg']==cthetadeg)
        &(run_dir_df['cinit']==cinit)
        &(run_dir_df['ithetadeg']==ithetadeg)
        &(run_dir_df['trial']==trial)]

    run_dir=run_dir_df_filtered['run_dir'][0]
    params_dict=get_params_dict(run_dir)
    data,adj=load_data(run_dir)
    data=find_images(data,params_dict)
    data=get_theta6_psi6(data,adj,run_dir)

    fig=plt.figure(figsize=(6.5,6.5),dpi=400)
    ax=fig.add_subplot(111)
    plot_sector(data,params_dict,ax,c='coordnum',cmap='newcmp')
    ax.set_ylim(min(data['y'])-10,min(data['y'])+reference_range+50)    
    ax.invert_yaxis() #setting ylim flips the axis..so we gotta flip it back..
    
    Nactual=len(data)
    info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)
    plt.savefig('analysis/cthetadeg'+str(cthetadeg)+'.svg',bbox_inches = "tight")

In [ ]:
fig=plt.figure(figsize=(1,1),dpi=400)
ax=fig.add_subplot(111)

x=np.arange(-3.5,-0.5,1)
y=np.repeat(np.sqrt(3)/2,len(x)) #row 1, left
ax.scatter(x,y,c='0.8')

x=np.arange(-3,0,1)
y=np.repeat(0,len(x)) #row 2, left
ax.scatter(x,y,c='0.8')

x=np.arange(-3.5,-0.5,1)
y=np.repeat(-np.sqrt(3)/2,len(x)) #row 3, left
ax.scatter(x,y,c='0.8')

shift=0.4

x=np.arange(-0.5+shift,2.5+shift,1)
y=np.repeat(np.sqrt(3)/2,len(x)) #row 1, right
ax.scatter(x,y,c='0.8')

x=np.arange(0+shift,2+shift,1)
y=np.repeat(0,len(x)) #row 2, right
ax.scatter(x,y,c='0.8')

x=np.arange(-0.5+shift,2.5+shift,1)
y=np.repeat(-np.sqrt(3)/2,len(x)) #row 3, right
ax.scatter(x,y,c='0.8')

dark_color=(0.1,0.2,0.6,1)
light_color=(0.1,0.2,0.6,0.5)

x=np.arange(0+shift,2+shift,1)[0] #left particle
y=np.repeat(np.sqrt(3),1) #row 0, right
ax.scatter(x,y,color=dark_color,s=mpl.rcParams['lines.markersize']**2) #default size
ax.scatter(x,y,c='1',s=mpl.rcParams['lines.markersize']**2-10) #draw outline (the built in edge thickness doesn't work)

x=np.arange(0+shift,2+shift,1)[1] #right particle
y=np.repeat(np.sqrt(3),1) #row 0, right
ax.scatter(x,y,color=light_color,s=mpl.rcParams['lines.markersize']**2) #default size
ax.scatter(x,y,c='1',s=mpl.rcParams['lines.markersize']**2-10) #draw outline (the built in edge thickness doesn't work)

x=np.arange(0+shift-1,2+shift,1)[0] #right particle
y=np.repeat(np.sqrt(3),1) #row 0, right
ax.scatter(x,y,color=light_color,s=mpl.rcParams['lines.markersize']**2) #default size
ax.scatter(x,y,c='1',s=mpl.rcParams['lines.markersize']**2-10) #draw outline (the built in edge thickness doesn't work)

ax.axis('off')
ax.set_xlim((-4,4))
ax.set_ylim((-4,4))
plt.savefig('analysis/growth_a.svg',bbox_inches = "tight")    
plt.show()


In [ ]:
fig=plt.figure(figsize=(1,1),dpi=400)
ax=fig.add_subplot(111)

x=np.arange(-3.5,-0.5,1)
y=np.repeat(np.sqrt(3)/2,len(x)) #row 1, left
ax.scatter(x,y,c='0.8')

x=np.arange(-3,0,1)
y=np.repeat(0,len(x)) #row 2, left
ax.scatter(x,y,c='0.8')

x=np.arange(-3.5,-0.5,1)
y=np.repeat(-np.sqrt(3)/2,len(x)) #row 3, left
ax.scatter(x,y,c='0.8')

#############
shift=0.4

x=np.arange(-0.5+shift,2.5+shift,1)
y=np.repeat(np.sqrt(3)/2,len(x)) #row 1, right
ax.scatter(x,y,c='0.8')

x=np.arange(0+shift,2+shift,1)
y=np.repeat(0,len(x)) #row 2, right
ax.scatter(x,y,c='0.8')

x=np.arange(-0.5+shift,2.5+shift,1)
y=np.repeat(-np.sqrt(3)/2,len(x)) #row 3, right
ax.scatter(x,y,c='0.8')
#############
dark_color=(0.8,0.2,0.2,1)
light_color=(0.8,0.2,0.2,0.5)
midpoint=(-1.5+(-0.5+shift))/2
x_defect=midpoint
y_defect=np.sqrt(1**2-(-0.5+shift-midpoint)**2)+np.sqrt(3)/2
ax.scatter(x_defect,y_defect,color=dark_color,s=mpl.rcParams['lines.markersize']**2) #default size
ax.scatter(x_defect,y_defect,c='1',s=mpl.rcParams['lines.markersize']**2-10) #draw outline (the built in edge thickness doesn't work)


pointA=np.array([x_defect,y_defect])
pointB=np.array([-0.5+shift,np.sqrt(3)/2]) #row 1, right
theta=np.deg2rad(60)
rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
v = np.dot(rot, pointB-pointA)
x=v[0]+pointA[0]
y=v[1]+pointA[1]
ax.scatter(x,y,color=light_color,s=mpl.rcParams['lines.markersize']**2) #default size
ax.scatter(x,y,c='1',s=mpl.rcParams['lines.markersize']**2-10) #draw outline (the built in edge thickness doesn't work)

pointA=np.array([x_defect,y_defect])
pointB=np.array([-1.5,np.sqrt(3)/2]) #row 1, left
theta=np.deg2rad(-60)
rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
v = np.dot(rot, pointB-pointA)
x=v[0]+pointA[0]
y=v[1]+pointA[1]
ax.scatter(x,y,color=light_color,s=mpl.rcParams['lines.markersize']**2) #default size
ax.scatter(x,y,c='1',s=mpl.rcParams['lines.markersize']**2-10) #draw outline (the built in edge thickness doesn't work)

ax.axis('off')
ax.set_xlim((-4,4))
ax.set_ylim((-4,4))
plt.savefig('analysis/growth_b.svg',bbox_inches = "tight")    
plt.show()

In [ ]:
axis_color='0.3'
defect_density_collect['cthetadeg']=np.array(defect_density_collect['cthetadeg']).astype(int) #because it bothers me that the values have decimals lol

cthetadeg_list=[5]
cinit=10
ithetadeg=40
trial=0
psi6cutoff_list=[0.5,0.6,0.7,0.8,0.9]

collect=defect_density_collect[(defect_density_collect['cthetadeg']>=5)&(defect_density_collect['cthetadeg']<=30)]
collectlabel='defect density'
for cthetadeg in cthetadeg_list:
    fig=plt.figure(figsize=(5,2.5),dpi=400)
    ax=fig.add_subplot(111)
    fixed1key='cinit'
    fixed1val=cinit
    fixed2key='cthetadeg'
    fixed2val=cthetadeg
    fixed3key='ithetadeg'
    fixed3val=ithetadeg

    legendvarkey='psi6cutoff'
    bin_parameter='circ'
    n=len(np.unique(collect[legendvarkey]))
    palette=psi_palette[1:1+n]
    title,df_sns=collect_plot_sns(ax,collectlabel,collect,fixed1key,fixed1val,fixed2key,fixed2val,fixed3key,fixed3val,legendvarkey,bin_parameter,palette)
    ax.set_ylabel(r'$\rho$')
    ax.set_xlabel(r'$C$')
    
    handles,labels = ax.get_legend_handles_labels()
    new_labels_list=[]
    for label in labels:
        new_labels_list+=[r'$\left|\psi_6\right|$<'+str(label)]
    new_labels = new_labels_list
    ax.legend(handles[::-1],new_labels[::-1],labelcolor='linecolor',loc='upper right',frameon=False,handlelength=0, handletextpad=0) #handlelength=0, handletextpad=0 removes the legend markers
    
    ax.set_xlim(0,12)
    ax.set_ylim(0,1)    
    
    ax.tick_params(direction="in",colors=axis_color)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['left'].set_color(axis_color)
    ax.xaxis.label.set_color(axis_color)
    ax.yaxis.label.set_color(axis_color)    
    
    Nactual=len(data)
    info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)+'_psi6cutoff-'+str(psi6cutoff)
    plt.savefig('analysis/psi6cutoff.svg',bbox_inches = "tight")

In [ ]:
axis_color='0.3'

cylinder_palette_sliced=sns.color_palette(
    c_palette[:n_cylinder][0:1]+ #C=5
    c_palette[:n_cylinder][2:3]+ #C=6 
    c_palette[:n_cylinder][4:5]+ #C=7
    c_palette[:n_cylinder][6:7]+ #C=8
    c_palette[:n_cylinder][8:9]+ #C=9
    c_palette[:n_cylinder][10:11] #C=10                           
)
cthetadeg=5
cinit_list=[5]
ithetadeg=40
trial=0
psi6cutoff=0.9

collect=defect_density_collect[(defect_density_collect['cthetadeg']>=5)&(defect_density_collect['cthetadeg']<=30)]
collect_list=[defect_density_collect[(defect_density_collect['cinit']==5)],
             defect_density_collect[(defect_density_collect['cinit']==6)],
             defect_density_collect[(defect_density_collect['cinit']==7)],
             defect_density_collect[(defect_density_collect['cinit']==8)],              
             defect_density_collect[(defect_density_collect['cinit']==9)],
             defect_density_collect[(defect_density_collect['cinit']==10)]]
collect=pd.concat(collect_list,ignore_index=True)
collect['cinit']=np.array(collect['cinit']).astype(int) #because it bothers me that the values have decimals lol
collectlabel='defect density'

for cinit in cinit_list:
    fig=plt.figure(figsize=(5,2.5),dpi=400)
    ax=fig.add_subplot(111)
    fixed1key='cthetadeg'
    fixed1val=cinit
    fixed2key='psi6cutoff'
    fixed2val=psi6cutoff
    fixed3key='ithetadeg'
    fixed3val=ithetadeg

    legendvarkey='cinit'
    bin_parameter='circ'
    palette=cylinder_palette_sliced
    
    title,df_sns=collect_plot_sns(ax,collectlabel,collect,fixed1key,fixed1val,fixed2key,fixed2val,fixed3key,fixed3val,legendvarkey,bin_parameter,palette)
    ax.set_ylabel(r'$\rho$')
    ax.set_xlabel(r'$C$')
    handles,labels = ax.get_legend_handles_labels()
    new_labels_list=[]
    for label in labels:
        new_labels_list+=[r'$C$='+str(label)] #r'$^{\circ}$'
    new_labels = new_labels_list
    ax.legend(handles[::-1],new_labels[::-1],labelcolor='linecolor',loc='upper right',frameon=False,handlelength=0, handletextpad=0) #handlelength=0, handletextpad=0 removes the legend markers
    ax.set_xlim(0,12)
    ax.set_ylim(0,1)    
    
    ax.tick_params(direction="in",colors=axis_color)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_color(axis_color)
    ax.spines['left'].set_color(axis_color)
    ax.xaxis.label.set_color(axis_color)
    ax.yaxis.label.set_color(axis_color)    
    
    Nactual=len(data)
    info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)+'_psi6cutoff-'+str(psi6cutoff)
    plt.savefig('analysis/cinit.svg',bbox_inches = "tight")

In [ ]:
cthetadeg_list=[5]
cinit_list=[5,6,7,8,9,10]
ithetadeg=40
trial=0

for cinit in cinit_list:
    run_dir_df_filtered=run_dir_df[
        (run_dir_df['cthetadeg']==cthetadeg)
        &(run_dir_df['cinit']==cinit)
        &(run_dir_df['ithetadeg']==ithetadeg)
        &(run_dir_df['trial']==trial)]

    run_dir=run_dir_df_filtered['run_dir'][0]
    params_dict=get_params_dict(run_dir)
    data,adj=load_data(run_dir)
    data=find_images(data,params_dict)
    data=get_theta6_psi6(data,adj,run_dir)

    fig=plt.figure(figsize=(6.5,6.5),dpi=400)
    ax=fig.add_subplot(111)
    plot_sector(data,params_dict,ax,c='coordnum',cmap='newcmp')
    
    ax.set_ylim(min(data['y'])-10,min(data['y'])+reference_range+50)    
    ax.invert_yaxis()
    
    Nactual=len(data)
    info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)
    plt.savefig('analysis/cinit_'+str(cinit)+'.svg',bbox_inches = "tight")    

    plt.show()

Load next dataset, 022723 bennett_datfiles.zip, which is used to generate S5.

In [ ]:
input_dir_c=parent_directory+'dataverse_files/022723 bennett_datfiles.zip' #near-commensurate dataset at ithetadeg=59deg
simulation_input_c='run_dir_list_022723_0-99.npy'
run_dir_list_c=np.unique(unzip_input(simulation_input_c))

print('get run_dir_df_c...') # separately import the new cylinder data which is in a different directory. note that run_dir is the full direct path for the cylinder df
run_dir_df_c=get_run_dir_df(run_dir_list_c)
run_dir_df_info(run_dir_df_c,input_dir=input_dir_c,output_dir=os.getcwd()+'/analysis')
run_dir_df_c.head()

print('done!')

In [ ]:
def export_fig7cylinder_c_data(run_dir_df=run_dir_df_c):
    fig7cylinder_data_list=[]
    counter=0
    start=time.time()

    cthetadeg=0
    ithetadeg_list=np.unique(run_dir_df['ithetadeg'])
    trials_list=np.unique(run_dir_df['trial'])
    cinit_list=np.unique(run_dir_df['cinit'])
    psi6cutoff_list=[0.9]
    for psi6cutoff in psi6cutoff_list:
        print('psi6cutoff ',psi6cutoff,end='...')
        for ithetadeg in ithetadeg_list:
            print('ithetadeg ',ithetadeg,end='...')

            fig7cylinder_data_list=[]
            for cinit in cinit_list:
                counter+=1
                print('cinit ',counter,'/',len(cinit_list),end='...')
                for trial in trials_list:
                    print('trial ',trial,end='...')

                    run_dir_df_filtered=run_dir_df[
                        (run_dir_df['cthetadeg']==cthetadeg)
                        &(run_dir_df['cinit']==cinit)
                        &(run_dir_df['ithetadeg']==ithetadeg)
                        &(run_dir_df['trial']==trial)]

                    run_dir=run_dir_df_filtered['run_dir'][0]
                    params_dict=get_params_dict(run_dir)
                    data,adj=load_data(run_dir)
                    data=find_images(data,params_dict)
                    data=get_theta6_psi6(data,adj,run_dir)

                    Ndefect=len(data[data['psi6']<psi6cutoff])
                    Nactual=len(data['psi6'])
                    defectdensity=Ndefect/Nactual

                    fig7cylinder_dict={'cthetadeg':cthetadeg,'cinit':cinit,'ithetadeg':ithetadeg,'trial':trial,'avg_psi6':np.mean(data['psi6']),
                               'psi6cutoff':psi6cutoff,'Ndefect':Ndefect,'Nactual':Nactual,'defectdensity':defectdensity}
                    fig7cylinder_data=pd.DataFrame([fig7cylinder_dict])
                    fig7cylinder_data_list+=[fig7cylinder_data]
            fig7cylinder_data=pd.concat(fig7cylinder_data_list)
            print('done!',round((time.time()-start)/60,2),'min to process')
            
            print('exporting fig7cylinder_c_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'_cinit0-12'+'.pkl...')
            start=time.time()
            fig7cylinder_data.to_pickle(output_dir+'/fig7cylinder_c_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'_cinit0-12'+'.pkl')
            print('done!',round((time.time()-start)/60,2),'min export')
def compile_fig7_c_data(run_dir_df=run_dir_df_c):
    fig7cylinder_data_list=[] #find defect density of cylinders. this is because my existing find_defect_density function doesn't work for cylinders (it assumed everything was able to be parametrized by r back then)
    start=time.time()

    cthetadeg=0
    ithetadeg_list=np.unique(run_dir_df['ithetadeg'])
    psi6cutoff_list=[0.9]    
    
    start=time.time()
    for psi6cutoff in psi6cutoff_list:
        for ithetadeg in ithetadeg_list:
            if os.path.exists('fig7cylinder_c_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'_cinit0-12'+'.pkl'):
                print('opening','fig7cylinder_c_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'_cinit0-12'+'.pkl','...')
                with open('fig7cylinder_c_data_ithetadeg-'+str(ithetadeg)+'_psi6cutoff-'+str(psi6cutoff)+'_cinit0-12'+'.pkl', "rb") as fh:
                    fig7cylinder_data_list+=[pickle.load(fh)]                    
    fig7cylinder_data=pd.concat(fig7cylinder_data_list)
    print('done!',round((time.time()-start)/60,2),'min import')

    print('exporting fig7cylinder_c_data.pkl...')
    start=time.time()
    fig7cylinder_data.to_pickle('fig7cylinder_c_data.pkl')
    print('done!',round((time.time()-start)/60,2),'min export')
def import_fig7cylinder_c_data():
    print('opening fig7cylinder_c_data.pkl...')
    start=time.time()
    with open('fig7cylinder_c_data.pkl', "rb") as fh:
        fig7cylinder_data=pickle.load(fh)
    print('done!',round((time.time()-start)/60,2),'min import')        
    return fig7cylinder_data

if not os.path.exists('fig7cylinder_c_data.pkl'): #aggregated fig7cylinder.pkl
    if not os.path.exists('fig7cylinder_c_data_ithetadeg-59.0_psi6cutoff-0.9_cinit0-12.pkl'): #individual fig7cylinder files for a set of cinit
        export_fig7cylinder_c_data() #might take a few days to run while on cluster        
    compile_fig7_c_data() #compile fig7cylinder_data into fig7_data

fig7cylinder_c_data=import_fig7cylinder_c_data()

In [ ]:
def export_data_collect_c(run_dir_df=run_dir_df_c):
    for jobnum in np.arange(1,100+1,1):
        run_dir_df_filtered=run_dir_df[((run_dir_df['trial']>=((jobnum-1)*1))&(run_dir_df['trial']<(jobnum*1)))]
        data_collect=get_data_collect(run_dir_df_filtered)
        info_tag=run_dir_df_info(run_dir_df_filtered)
        data_collect.to_pickle('c_data_collect_'+info_tag+'.pkl') #prefix with c_ so easier to search
def compile_data_collect_c():
    start=time.time()
    counter=0
    pkl_file_list=[]
    for pkl_file in os.listdir(os.getcwd()): #get directory names in analysis folder
        if pkl_file.startswith('c_data_collect') and pkl_file.endswith('.pkl'):
            pkl_file_list+=[pkl_file]
            counter+=1
            print('.',end='')
    print('\n\n',counter,'files imported... loading c_data_collect...')
    data_collect_list=[]
    for pkl_file in pkl_file_list:
        with open(pkl_file, "rb") as fh: #read binary = rb, #file handle = fh
            data_collect = pickle.load(fh)
            data_collect_list += [data_collect]
            print('.',end='')
    data_collect=pd.concat(data_collect_list,ignore_index=True)
    print('done!',round((time.time()-start)/60,2),'min import')
    print('exporting c_data_collect_compiled.pkl...')
    start=time.time()
    data_collect.to_pickle('c_data_collect_compiled.pkl')
    print('done!',round((time.time()-start)/60,2),'min import')
def data_to_defect_collect_c():
    print('opening c_data_collect_compiled.pkl...')
    start=time.time()
    with open('c_data_collect_compiled.pkl', "rb") as fh:
        data_collect_compiled=pickle.load(fh)
    print('done!',round((time.time()-start)/60,2),'min import')

    data_collect=data_collect_compiled

    print('finding defect_density')
    psi6cutoff_list=[0.9] #psi6cutoff for defining a defect
    data_collect_psi6cutoff=find_data_collect_psi6cutoff(data_collect,psi6cutoff_list) #used for finding kde... kind of redundant with the defect_density code but whatever i don't want to rewrite it right now
    defect_density_collect,defect_count_collect,data_count_collect=get_defect_density_collect(data_collect,psi6cutoff_list)
    print('data_collect pkl input:\n','data_collect_compiled.pkl')
    print('\n imported data_collect pickle file. \n found data_collect_psi6cutoff, \n defect_density_collect, defect_count_collect, data_count_collect for',len(np.unique(data_collect['trial'])),'trials at each parameter set \n\n')
    defect_density_collect_dict={'trials':len(np.unique(data_collect['trial']))}
    for item in defect_density_collect.columns[:5]:
        print(item,'\n',np.unique(defect_density_collect[item]))
        defect_density_collect_dict.update({item:np.unique(defect_density_collect[item])})

    print('\ndefect_density_collect_dict\n',defect_density_collect_dict,'\n')

    defect_density_collect.head()

    print('exporting defect_density_collect_c.pkl...')
    start=time.time()
    defect_density_collect.to_pickle('defect_density_collect_c.pkl')
    print('done!',round((time.time()-start)/60,2),'min export')
def import_collect_pkl_c():
    print('opening defect_density_collect_c.pkl...')
    start=time.time()
    with open(output_dir+'/defect_density_collect_c.pkl', "rb") as fh:
        defect_density_collect_c=pickle.load(fh)
    print('done!',round((time.time()-start)/60,2),'min import')
    return defect_density_collect_c

if os.path.exists('defect_density_collect_c.pkl.zip'): #intermediate analysis file of dataset
    defect_density_collect_c=unzip_input('defect_density_collect_c.pkl.zip')
    print('imported defect_density_collect_c.pkl.zip as defect_density_collect_c')
elif os.path.exists('defect_density_collect_c.pkl'): #intermediate analysis file of dataset
    defect_density_collect_c=unzip_input('defect_density_collect_c.pkl')
    print('imported defect_density_collect_c.pkl as defect_density_collect_c')    
else: #warning: the following steps will take more than a few days on a cluster to run.
    export_data_collect_c() #generates and exports data_collect for each trial number 
    compile_data_collect_c() #generates and exports a large, compiled data_collect
    data_to_defect_collect_c() #converts data_collect to defect_density_collect, and exports
    defect_density_collect_c=import_collect_pkl_c() #reimport defect_density_collect
    print('generated and imported defect_density_collect_c.pkl as defect_density_collect_c')

In [ ]:
cone_cylinder_df=pd.DataFrame([])
start=time.time()
collect=defect_density_collect_c[(defect_density_collect_c['bin_parameter']=='circ')
                               &(defect_density_collect_c['cthetadeg']>0)&(defect_density_collect_c['cthetadeg']<=30)]
cone_df_list=[]
for cthetadeg in np.unique(collect['cthetadeg']):
    for ithetadeg in np.unique(collect['ithetadeg']):
        for cinit in np.unique(collect['cinit']):
            for psi6cutoff in np.unique(collect['psi6cutoff']):
                collect_filtered=collect[(collect['cthetadeg']==cthetadeg)&(collect['ithetadeg']==ithetadeg)&(collect['cinit']==cinit)&(collect['psi6cutoff']==psi6cutoff)].dropna(axis=1)
                defect_density=collect_filtered.iloc[:,5:-1].values[0]
                bin_edges=collect_filtered.iloc[:,5:].columns.values
                if len(bin_edges)>1:
                    midpoint=(bin_edges[1]-bin_edges[0])/2 #since bins are from r1 to r2 or c1 to c2; find r0 or c0, which is the midline of the frustum
                    cone_df=pd.DataFrame([])                
                    cone_df['circ']=bin_edges[:-1]+midpoint
                    cone_df['defect_density']=defect_density
                    cone_df['cthetadeg']=cthetadeg
                    cone_df['ithetadeg']=ithetadeg
                    cone_df['cinit']=cinit
                    cone_df['psi6cutoff']=psi6cutoff
                    cone_df_list+=[cone_df]
cone_df=pd.concat(cone_df_list)

cylinder_df_list=[]
cylinder_df=fig7cylinder_c_data #find average defect density for all trials
mean_cylinder_df_list=[]
for cthetadeg in np.unique(cylinder_df['cthetadeg']):            
    for ithetadeg in np.unique(cylinder_df['ithetadeg']):            
        for cinit in np.unique(cylinder_df['cinit']):                        
            for psi6cutoff in np.unique(cylinder_df['psi6cutoff']):                                        
                mean_defect_density=np.mean(cylinder_df[(cylinder_df['cthetadeg']==cthetadeg)&(cylinder_df['ithetadeg']==ithetadeg)
                &(cylinder_df['cinit']==cinit)&(cylinder_df['psi6cutoff']==psi6cutoff)]['defectdensity'])
                mean_cylinder_df=pd.DataFrame([{'defect_density':mean_defect_density,'cthetadeg':cthetadeg,'ithetadeg':ithetadeg,'cinit':cinit,'psi6cutoff':psi6cutoff}])
                mean_cylinder_df_list+=[mean_cylinder_df]
mean_cylinder_df=pd.concat(mean_cylinder_df_list)
for cinit in np.unique(collect['cinit']): #repeat cylinder values for all cinit (so that we can plot it with the cone values) - this means that cinit value won't really be physical for cylinders!!!
    cylinder_df=pd.DataFrame([])
    cylinder_df['circ']=mean_cylinder_df['cinit'] #CYLINDER cinit.
    cylinder_df['defect_density']=mean_cylinder_df['defect_density']
    cylinder_df['cthetadeg']=mean_cylinder_df['cthetadeg']
    cylinder_df['ithetadeg']=mean_cylinder_df['ithetadeg']
    cylinder_df['cinit']=cinit #CONE cinit. not the actual cylinder cinit.
    cylinder_df['psi6cutoff']=mean_cylinder_df['psi6cutoff']
    cylinder_df_list+=[cylinder_df]
cylinder_df=pd.concat(cylinder_df_list)
cone_cylinder_df=pd.concat([cone_df,cylinder_df])

print(round((time.time()-start)/60,2),'min to process')

In [ ]:
def find_coeff_df(cone_cylinder_df,xcutoff_offset,xcutoff_max,ithetadeg_list,cinit_list,psi6cutoff_list): #find coefficients for best exponential fit
    coeff_df_list=[]
    for ithetadeg in ithetadeg_list:
        for cinit in cinit_list:
            for psi6cutoff in psi6cutoff_list:
                cone_cylinder_df_filtered=cone_cylinder_df[(cone_cylinder_df['ithetadeg']==ithetadeg)&(cone_cylinder_df['psi6cutoff']==psi6cutoff)&(cone_cylinder_df['cinit']==cinit)]
                df_sns=pd.DataFrame([])
                df_sns['x']=cone_cylinder_df_filtered['circ']
                df_sns['y']=cone_cylinder_df_filtered['defect_density']
                df_sns['c']=cone_cylinder_df_filtered['cthetadeg']

                xcutoff_offset=xcutoff_offset
                xcutoff_max=xcutoff_max
                for cvalue in np.unique(df_sns['c']): #find fits for each dataset
                    df_sns_filtered=df_sns[(df_sns['c']==cvalue)&(df_sns['y']>=0.00001)] #select dataset ('c'). y needs to be nonzero.

                    ymax=max(df_sns_filtered['y'])
                    xcutoff_left=df_sns_filtered[df_sns_filtered['y']==ymax]['x'].to_numpy()[0]+xcutoff_offset
                    df_sns_filtered=df_sns_filtered[df_sns_filtered['x']>=xcutoff_left]

                    ymin=min(df_sns_filtered['y'])
                    xcutoff_right=df_sns_filtered[df_sns_filtered['y']==ymin]['x'].to_numpy()[0]
                    if xcutoff_max[0]==True:
                        xcutoff_right=xcutoff_max[1]
                    df_sns_filtered=df_sns_filtered[df_sns_filtered['x']<=xcutoff_right]

                    x_data=df_sns_filtered['x'].astype(float)
                    y_data=df_sns_filtered['y'].astype(float)

                    log_y_data = np.log(y_data)
                    a,b = np.polyfit(x_data, log_y_data, 1) #fit line to log(y) vs. x data, gives straight line if exponential relationship between y and x
                    y=np.exp(a*x_data+b)
                    coeff_df=pd.DataFrame([])
                    coeff_df['circ_filtered']=x_data
                    coeff_df['defect_density']=y_data
                    coeff_df['defect_density_fit']=y
                    coeff_df['a']=a #y = exp(a*x+b)
                    coeff_df['b']=b #y = exp(a*x+b)
                    coeff_df['psi6cutoff']=psi6cutoff
                    coeff_df['ithetadeg']=ithetadeg
                    coeff_df['cthetadeg']=cvalue
                    coeff_df['cinit']=cinit            
                    coeff_df_list+=[coeff_df]
    coeff_df=pd.concat(coeff_df_list) 
    return coeff_df

cinit_list=[10]
psi6cutoff_list=[0.9]
ithetadeg_list=np.unique(cone_cylinder_df[cone_cylinder_df['cthetadeg']==0]['ithetadeg'])
xcutoff_offset=1 #offset calculated left cutoff value
xcutoff_max=[True,10] #right cutoff at fixed value
coeff_df=find_coeff_df(cone_cylinder_df,xcutoff_offset,xcutoff_max,ithetadeg_list,cinit_list,psi6cutoff_list) #find coefficients for best exponential fit

cthetadeg_list=np.unique(coeff_df['cthetadeg'])
for cthetadeg in cthetadeg_list:
    print('cthetadeg: '+str(cthetadeg),'| left cutoff: '+str(round(min(coeff_df[coeff_df['cthetadeg']==cthetadeg]['circ_filtered']),2)),
          '| right cutoff: '+str(round(max(coeff_df[coeff_df['cthetadeg']==cthetadeg]['circ_filtered']),2)))

In [ ]:
cone_cylinder_df=cone_cylinder_df.reset_index(drop=True)

axis_color='0.3'
n_cone=6 #5,10,15...30
n_cylinder=11 #2,3,4,...,12
cylinder_palette=sns.color_palette(c_palette[:n_cylinder][6:7]) #select C=10
cone_palette=sns.color_palette(phi_palette[2:2+n_cone])

psi6cutoff=0.9
ithetadeg=59
cinit=10
cone_cylinder_df_filtered=cone_cylinder_df[(cone_cylinder_df['psi6cutoff']==psi6cutoff)&(cone_cylinder_df['ithetadeg']==ithetadeg)&(cone_cylinder_df['cinit']==cinit)] #orig data

fig=plt.figure(figsize=(5,2.5),dpi=400)
ax=fig.add_subplot(111)
cone=cone_cylinder_df_filtered[cone_cylinder_df_filtered['cthetadeg']!=0]
cone=cone.reset_index()
cylinder=cone_cylinder_df_filtered[cone_cylinder_df_filtered['cthetadeg']==0]

line_sizes=list(np.linspace(0.85,1.15,n_cone))
sns.scatterplot(x=cylinder['circ'], y=cylinder['defect_density'],color=cylinder_palette[0],ax=ax,legend=False,zorder=1) 
sns.lineplot(x=cone['circ'], y=cone['defect_density'],hue=cone['cthetadeg'],palette=cone_palette,ax=ax,legend=True,zorder=0,size=cone['cthetadeg'],sizes=line_sizes)    

handles,labels = ax.get_legend_handles_labels()
new_labels_list=[]
new_labels_list+=[r'$\phi$='+str(0)+r'$^{\circ}$'] #cylinder label
handles+=[handles[0]] #add an additional entry for cylinder
for label in labels:
    new_labels_list+=[r'$\phi$='+str(int(float(label)))+r'$^{\circ}$'] #cone labels
new_labels = new_labels_list
ax.legend(handles,new_labels,labelcolor=cylinder_palette+cone_palette,loc='upper right',frameon=False,handlelength=0,handletextpad=0,labelspacing=0)

xlimL,xlimR=0,12
ax.set_xlim(xlimL,xlimR)
ax.set_ylim(0,1)

title='defect density'+' vs '+'circ'+', ithetadeg='+str(ithetadeg)+'\npsi6cutoff='+str(0.9)+', cinit='+str(cinit)+'\n'
ax.set_title(title+'\n')
ax.set_ylabel(r'$\rho$');
ax.set_xlabel(r'$C$');        
ax.annotate('increasing '+r'$\phi$',xy=(5, 0.1), xycoords='data',xytext=(6, 0.5), textcoords='data',arrowprops=dict(arrowstyle="->",connectionstyle="arc3,rad=-0.2",color=axis_color),color=axis_color,)

ax.tick_params(direction="in",colors=axis_color)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_color(axis_color)
ax.spines['left'].set_color(axis_color)
ax.xaxis.label.set_color(axis_color)
ax.yaxis.label.set_color(axis_color)
plt.savefig('analysis/cylinderplot_c.svg',bbox_inches = "tight")
plt.show()

In [ ]:
input_dir=input_dir_c
cthetadeg=0
ithetadeg=59
trial=0

cinit_list=[2,4,6,8,10,12]

ref_ylim_range=reference_range+50+10 #this is the standard range used for all of my plots so far
new_ylim_range=reference_range+5

ylim_range_ratio=new_ylim_range/ref_ylim_range

for cinit in cinit_list:
    run_dir_df_filtered=run_dir_df_c[
        (run_dir_df_c['cthetadeg']==cthetadeg)
        &(run_dir_df_c['cinit']==cinit)
        &(run_dir_df_c['ithetadeg']==ithetadeg)
        &(run_dir_df_c['trial']==trial)]

    run_dir=run_dir_df_filtered['run_dir'][0]
    params_dict=get_params_dict(run_dir)
    data,adj=load_data(run_dir)
    data=find_images(data,params_dict)
    data=get_theta6_psi6(data,adj,run_dir)

    fig=plt.figure(figsize=(6.5,6.5*ylim_range_ratio),dpi=400)
    ax=fig.add_subplot(111)
    plot_sector(data,params_dict,ax,c='coordnum',cmap='newcmp')
    
    ylim_left=min(data['y'])-10
    ax.set_ylim(ylim_left,ylim_left+new_ylim_range) #the max is always y=20 above the nucleation seed. so we really only need to find the lower bound    
    ax.invert_yaxis() #setting ylim flips the axis..so we gotta flip it back..
    
    Nactual=len(data)
    info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)
    plt.savefig('analysis/cylinder_cinit'+str(cinit)+'_c.svg',bbox_inches = "tight")
    plt.show()

Finally, load last dataset, 022323 bennett simulation cthetadeg0 magic mn_datfiles.zip, which is used to generate Fig S4 B. Fig S4 A can be found in magic.nb, a Mathematica notebook.

In [ ]:
input_dir_m=parent_directory+'dataverse_files/022323 bennett simulation cthetadeg0 magic mn_datfiles.zip' #magic angle cylinders demonstrating that Bennett can achieve perfect packings even at chiral orientation angles
simulation_input_m='run_dir_list_022323_0-2cinit0-12.npy'

run_dir_list_m=np.unique(unzip_input(simulation_input_m))

print('get run_dir_df_m...') # separately import the new cylinder data which is in a different directory. note that run_dir is the full direct path for the cylinder df
run_dir_df_m=get_run_dir_df(run_dir_list_m)
run_dir_df_info(run_dir_df_m,input_dir=input_dir_m,output_dir=os.getcwd()+'/analysis')
run_dir_df_m.head()

print('done!')

In [ ]:
def get_cylinder_cinit(m,n,a): #find commensurate cylinder cinit for a given initial orientation
    cinit=a*np.sqrt(m**2+n**2-(m*n))
    return cinit
def get_cylinder_ithetadeg(m,n):
    mn_ratio=m/n
    phi_dbeller=np.arctan((2/np.sqrt(3))*((mn_ratio)-(1/2))) #phi based on DA Beller, DR Nelson convention (angle made with vertical axis)
    phi_dbeller_deg=phi_dbeller*180/np.pi
    ithetadeg=(90-phi_dbeller_deg) #theta based on our convention, angle made with circumference
    return phi_dbeller_deg,ithetadeg
def get_cylinder_params(m,n,a): #characterize in terms of parastichy numbers
    phi_dbeller_deg,ithetadeg=get_cylinder_ithetadeg(m,n)
    cinit=get_cylinder_cinit(m,n,a)
    mn_ratio=m/n
    cylinder_params_df=pd.DataFrame([{'m':m,'n':n,'mn_ratio':mn_ratio,'phi_dbeller_deg':phi_dbeller_deg,'ithetadeg':ithetadeg,'a':a,'cinit':cinit}])
    return cylinder_params_df

a_list=np.arange(1,11,1)
cylinder_params_df_list=[]
mn_list=[(1,1),(2,1),(3,2),(4,3),(5,4)]
for mn in mn_list:
    m,n=mn[0],mn[1]
    for a in a_list:
        cylinder_params_df=get_cylinder_params(m,n,a)
        cylinder_params_df_list+=[cylinder_params_df]    
cylinder_params_df=pd.concat(cylinder_params_df_list)
cylinder_params_df=cylinder_params_df[cylinder_params_df['cinit']<=12] #from experience i know that simulations break down at high circumferences due to numerical error
cylinder_params_df.to_pickle('cylinder_params_df.pkl') #relates cylinder parameters to parastichy numbers (m,n)

In [ ]:
run_dir_df=run_dir_df_m
input_dir=input_dir_m
run_dir_df_filtered=run_dir_df[(run_dir_df['trial']==0)&(run_dir_df['ithetadeg']>=40)&(run_dir_df['cinit']>=2.5)&(run_dir_df['cinit']<=8)] #select only a small range
for run_dir in run_dir_df_filtered['run_dir'].tolist()[::-1]:
    run_dir_params=run_dir_df[run_dir_df['run_dir']==run_dir]
    cthetadeg=run_dir_params['cthetadeg'].values[0]
    cinit=run_dir_params['cinit'].values[0]
    ithetadeg=run_dir_params['ithetadeg'].values[0]
    trial=run_dir_params['trial'].values[0]

    magic_info=cylinder_params_df[(cylinder_params_df['cinit']==cinit)] #each cylinder plot can be indexed by (C, theta) and trial. theta stores (m,n) info for parastichy number.
    if ithetadeg==magic_info['ithetadeg'].values[0]:
        params_dict=get_params_dict(run_dir)
        data,adj=load_data(run_dir,input_dir=input_dir)
        data['x3D'],data['y3D'],data['z3D']=uv_wrap(data['x'],data['y'],params_dict['cthetadeg']*np.pi/180)

        reference_range=40 #temporary set
        fig=plt.figure(figsize=(6.5,3),dpi=400)
        ax=fig.add_subplot(121)
        plot_sector(data,params_dict,ax,cmap=cmap)
        Nactual=len(data)
        info_tag='_cthetadeg-'+str(cthetadeg)+'_cinit-'+str(cinit)+'_ithetadeg-'+str(ithetadeg)+'_trial-'+str(trial)+'_Nactual-'+str(Nactual)
        ax.set_ylim(min(data['y'])-10,min(data['y'])+reference_range+50)
        ax.invert_yaxis()
        ax1=fig.add_subplot(122,projection='3d')
        plot_cone(data,params_dict,ax1,azim_val=180,cmap=cmap)
        ax1.set_zlim(max(data['z3D'])-reference_range-50,max(data['z3D'])+10)        
        suptitle='C='+str(round(cinit,3))+', '+r'$\theta_{seed}$='+str(round(ithetadeg,3))+r'$^\circ$'+'\n'
        
        c=str(round(cinit,3))
        theta_seed=str(round(ithetadeg,3))+r'$^\circ$'
        mn='('+str(magic_info['m'].values[0])+','+str(magic_info['n'].values[0])+')'
        a=str(magic_info['a'].values[0])
        phi_seed=str(round(magic_info['phi_dbeller_deg'].values[0],3))+r'$^\circ$'
        suptitle='C='+c+', '+r'$\theta_{seed}$='+theta_seed+'\n'+'(m,n)='+mn+', a='+a+', '+r'$\phi_{seed}$='+phi_seed
        
        plt.suptitle(suptitle)
        fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
    
        plt.savefig('analysis/C-'+str(round(cinit,3))+'_theta-'+str(round(ithetadeg,3))+'.png',bbox_inches = "tight")
        plt.show()